In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
pd.set_option('display.max_columns', None)

## Import Data Dictionary

In [2]:
dictionary = pd.read_csv("DATADIC.csv")
dictionary.head()

,Phase,FLDNAME,TBLNAME,CRFNAME,TEXT,TYPE,LENGTH,CODE,UNITS,status
0,ADNI1,ID,AAL,NaN,Record ID,N,38,"""crfname"","""",""indexes"",""adni_aal_idx=TBLID,FLD...",NaN,NaN
1,ADNI1,SX,AAL,NaN,NaN,N,10,NaN,NaN,NaN
2,ADNI1,USERID,AAL,NaN,NaN,T,100,NaN,NaN,NaN
3,ADNI1,USERDATE,AAL,NaN,Date record created,S,0,NaN,NaN,NaN
4,ADNI1,USERID2,AAL,NaN,NaN,T,100,NaN,NaN,NaN


## Data Overview

The main dataset (ADNIMERGE), that can be downloaded from the “study data” section, contains observations representing individual visits by subjects. Most columns represent a single biomarker. Some important columns are a **unique subject identifier (RID)** and **visit code (VISCODE)**, indicating the time of the visit (e.g. baseline, 6 month follow-up, etc.).

There are two different *initial visit codes*: 

- Diagnosis at the **screen visit** (VISCODE2=‘sc’):
  - When randomization is assigned
  - Determines visitation schedule, which is diagnosis specific
  - Available in the arm table (ARM.csv)

- Diagnosis at **baseline visit** (VISCODE2=‘bl’)
  - Based on additional information acquired at the baseline visit to obtain diagnosis
  - Diagnosis recommended by the Clinical Core for use as initial diagnosis
  - Available in the diagnosis summary table (DXSUM PDXCONV ADNIALL.csv)
  
The **screening** visit consists of a subset of the planned assessments (neuropsychological testing, imaging, etc.) sufficient to evaluate the inclusion/exclusion criteria for each potential participant. Once a participant passed screening, the **baseline assessment** was conducted in which additional neuropsychological testing, imaging, and fluid samples were taken. 
  
The `ADNIMERGE` file includes demographics, neuropsychological testing scores, MRI and PER summaries. It also includes both a baseline diagnosis and diagnosis at each visit. 
  
More info in: https://adni.loni.usc.edu/wp-content/uploads/2008/07/inst_about_data.pdf

General Procedures Manual: https://adni.loni.usc.edu/wp-content/uploads/2010/09/ADNI_GeneralProceduresManual.pdf

### ADAS Sub-Scores and Total Scores [ADNI1]

The difference between `TOTAL_11` and `TOTALMOD` is that the first one excludes Q4 and Q14. 

Raw data. 

I think that these are the total scores for each task contained in the ADAS [ADNI1] dataset. 

In [3]:
ADASSCORES = pd.read_csv("Neuropsychological/ADASSCORES.csv")
ADASSCORES[ADASSCORES['VISCODE']=='bl'].head()

,ID,RID,SITEID,VISCODE,USERDATE,USERDATE2,EXAMDATE,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,Q11,Q12,Q14,TOTAL11,TOTALMOD,update_stamp
2,16500121,1412,20,bl,2014-08-20,NaN,2007-07-30,3.67,0,1,3,0,0,0,8,0,0,1,0,1,13.67,17.67,2014-08-20 00:00:00.0
4,16500117,1411,20,bl,2014-08-20,NaN,2007-08-08,4.67,0,0,5,0,0,0,4,0,0,0,0,0,8.67,13.67,2014-08-20 00:00:00.0
7,16500111,1409,115,bl,2014-08-20,NaN,2007-07-30,9.00,0,3,10,0,2,3,5,3,3,4,1,-4,33.00,-4.00,2014-08-20 00:00:00.0
11,16500103,1408,115,bl,2014-08-20,NaN,2007-08-01,2.00,0,0,1,0,0,0,1,0,0,0,0,0,3.00,4.00,2014-08-20 00:00:00.0
15,16500095,1407,115,bl,2014-08-20,NaN,2007-06-25,4.67,0,1,6,0,0,0,7,0,0,1,1,0,14.67,20.67,2014-08-20 00:00:00.0


Number of patients

In [6]:
#Count number of patients
len(ADASSCORES[ADASSCORES['VISCODE']=='bl'])

819

In [90]:
ADASSCORES['VISCODE'].unique()

array(['m12', 'm06', 'bl', 'm24', 'm36', 'm18', 'm48', 'uns1'],
      dtype=object)

### ADSP Phenotype Harmonization Consortium (PHC) - Composite Cognitive Scores [ADNI 1, GO, 2,3] 

Neuropsychological batteries differ across studies and cohorts, complicating the ability to merge cognitive data across cohorts. **Co-calibrated composite scores** generated using modern psychometric approaches permit direct comparison of study participants in different studies who were assessed with different test batteries.

The `PHC_Diagnosis` column is the one that contains the diagnosis status for the patient:

- 1: normal cognition
- 2: mild cognitive impairment
- 3: Alzheimer's disease

It seems that the tests are: 

- `PHC_MEM`: harmonized composite memory score
- `PHC_EXF`: harmonized composite executive function score
- `PHC_LAN`: harmonized composite language score
- `PHC_VSP`: harmonized composite visuospatial score

"The harmonized domain-specific cognitive scores from each study were derived using modern psychometric modeling which produce scores and also the standard error of measurement (SEM) around those scores"

I think they use data from other test batteries such as MoCA, MMSE... to obtain the scores. 

There is more info in the methods PDF document. 

- `VISCODE`: visit code
- `VISCODE2`: translated visit code

In [71]:
ADSP = pd.read_csv("Neuropsychological/ADSP_PHC_COGN_10_05_22.csv")
ADSP[ADSP['VISCODE2']=='bl'].head()

,RID,SUBJID,PHASE,VISCODE,VISCODE2,EXAMDATE,PHC_VISIT,PHC_Sex,PHC_Education,PHC_Ethnicity,PHC_Race,PHC_AGE,PHC_Diagnosis,PHC_MEM,PHC_EXF,PHC_LAN,PHC_VSP,update_stamp
0,2,ADNI_011_S_0002,ADNI1,bl,bl,2005-09-08,1,1.0,16.0,2.0,5.0,74.439425,1.0,0.277,0.254,0.293,NaN,2022-10-18 09:15:45.0
11,3,G-ADNI-AN000439,ADNI1,bl,bl,2005-09-12,1,1.0,18.0,2.0,5.0,81.366188,3.0,-1.068,-0.670,-0.516,-0.237,2022-10-18 09:15:45.0
15,4,G-ADNI-AN000446,ADNI1,bl,bl,2005-11-08,1,1.0,10.0,1.0,5.0,67.852156,2.0,-0.014,-0.603,0.487,-0.074,2022-10-18 09:15:45.0
20,5,G-ADNI-AN000663,ADNI1,bl,bl,2005-09-07,1,1.0,16.0,2.0,5.0,73.768652,1.0,0.520,0.577,0.345,NaN,2022-10-18 09:15:45.0
25,6,G-ADNI-AN000679,ADNI1,bl,bl,2005-11-29,1,2.0,13.0,2.0,5.0,80.495551,2.0,-0.325,-0.195,0.065,-0.237,2022-10-18 09:15:45.0


In [89]:
#Count number of patients
len(ADSP[ADSP['VISCODE2']== 'bl'] ) + len(ADSP[ADSP['VISCODE2']== 'sc'])

3175

In [79]:
ADSP['VISCODE2'].unique()

array(['bl', 'm06', 'm36', 'm60', 'm72', 'm84', 'm96', 'm108', 'm120',
       'm132', 'm144', 'm12', 'm24', 'm18', 'm48', 'm156', 'm168', 'sc'],
      dtype=object)

In [10]:
ADSP_DICT = pd.read_csv("Neuropsychological/ADSP_PHC_COGN_DICT_10_05_22.csv")
ADSP_DICT

,ID,FLDNAME,TBLNAME,CRFNAME,QUESTION,TEXT,STATUS,DEPRULE,TYPE,LENGTH,DECIML,RANGEVAL,RANGESQL,RANGEPRL,UNITS,CODE,NOTES,OBJID,COLID,RANGEJSCPT,NAVJSCPT,FTYPE,TAB,DE,MC,VISIBLE,LOGVISIBLE,MTBLNAME,SUBTBLNAME,POPJSCPT,GRID,DEF,update_stamp
0,-4,RID,ADSP_PHC_COGN,ADSP Phenotype Harmonization Consortium (PHC) ...,-4,Participant roster ID,-4,-4,-4,-4,-4,-4,-4,-4,-4,-4,NaN,-4,-4,-4,-4,-4,-4,-4,-4,-4,-4,-4,-4,-4,-4,-4,2022-10-06 18:42:42.0
1,-4,SUBJID,ADSP_PHC_COGN,ADSP Phenotype Harmonization Consortium (PHC) ...,-4,Subject ID used in NIAGADS for the whole genom...,-4,-4,-4,-4,-4,-4,-4,-4,-4,-4,NaN,-4,-4,-4,-4,-4,-4,-4,-4,-4,-4,-4,-4,-4,-4,-4,2022-10-17 21:52:40.0
2,-4,PHASE,ADSP_PHC_COGN,ADSP Phenotype Harmonization Consortium (PHC) ...,-4,ADNI Phase,-4,-4,-4,-4,-4,-4,-4,-4,-4,-4,NaN,-4,-4,-4,-4,-4,-4,-4,-4,-4,-4,-4,-4,-4,-4,-4,2022-10-06 18:42:42.0
3,-4,VISCODE,ADSP_PHC_COGN,ADSP Phenotype Harmonization Consortium (PHC) ...,-4,Visit code,-4,-4,-4,-4,-4,-4,-4,-4,-4,-4,NaN,-4,-4,-4,-4,-4,-4,-4,-4,-4,-4,-4,-4,-4,-4,-4,2022-10-06 18:42:42.0
4,-4,VISCODE2,ADSP_PHC_COGN,ADSP Phenotype Harmonization Consortium (PHC) ...,-4,Translated visit code,-4,-4,-4,-4,-4,-4,-4,-4,-4,-4,NaN,-4,-4,-4,-4,-4,-4,-4,-4,-4,-4,-4,-4,-4,-4,-4,2022-10-06 18:42:42.0
5,-4,EXAMDATE,ADSP_PHC_COGN,ADSP Phenotype Harmonization Consortium (PHC) ...,-4,Examination Date,-4,-4,-4,-4,-4,-4,-4,-4,-4,-4,NaN,-4,-4,-4,-4,-4,-4,-4,-4,-4,-4,-4,-4,-4,-4,-4,2022-10-06 18:42:42.0
6,-4,PHC_Visit,ADSP_PHC_COGN,ADSP Phenotype Harmonization Consortium (PHC) ...,-4,Visit Code (derived - see NOTES),-4,-4,-4,-4,-4,-4,-4,-4,-4,-4,"Calculated from: RID and EXAMDATE, Given diffe...",-4,-4,-4,-4,-4,-4,-4,-4,-4,-4,-4,-4,-4,-4,-4,2022-10-17 21:52:40.0
7,-4,PHC_Age,ADSP_PHC_COGN,ADSP Phenotype Harmonization Consortium (PHC) ...,-4,Age at visit (derived - see NOTES),-4,-4,-4,-4,-4,-4,-4,-4,-4,-4,"Calculated from: EXAMDATE, PTDOBMM, and PTDOBY...",-4,-4,-4,-4,-4,-4,-4,-4,-4,-4,-4,-4,-4,-4,-4,2022-10-17 21:52:40.0
8,-4,PHC_MEM,ADSP_PHC_COGN,ADSP Phenotype Harmonization Consortium (PHC) ...,-4,Harmonized composite memory score (derived - s...,-4,-4,-4,-4,-4,-4,-4,-4,-4,-4,NaN,-4,-4,-4,-4,-4,-4,-4,-4,-4,-4,-4,-4,-4,-4,-4,2022-10-06 18:42:42.0
9,-4,PHC_EXF,ADSP_PHC_COGN,ADSP Phenotype Harmonization Consortium (PHC) ...,-4,Harmonized composite executive function score ...,-4,-4,-4,-4,-4,-4,-4,-4,-4,-4,NaN,-4,-4,-4,-4,-4,-4,-4,-4,-4,-4,-4,-4,-4,-4,-4,2022-10-06 18:42:42.0


### Alzheimer's Disease Assessment Scale (ADAS) [ADNI1] 

- **COT1SCOR**, **COT2SCOR**, **COT3SCOR** would be the sum of the number of words present in lists COT1LIST, COT2LIST and COT3LIST. ¿Each list is a trial?

- **COP1COMP**: If any trial not administer check one: ","N","1","1=Not done (for reasons other than physical/cognitive),  2=Participant refused, 4=Participant unable to complete 3 trials for cognitive reasons (NOTE: A score of '0' should be given in each of Trials 1, 2, and 3 if the task was administered and the participant was unable to complete the task for cognitive reasons.)","","" 
- **COCOMND**: Check each command performed correctly.","T","20","1= Make a fist. 2...
- **COP2COMP**: if task not administed check one...
- **COCONSTR**: Check each figure drawn correctly.
- **COT4LIST**: Check each word correctly recalled
- **CONAME**: Check each object and finger named correctly
- **COIDEA**: Check each step completed correctly
- **COORIEN**: Check each item answered correctly
- **CO8xx**: Yes or No 
- **CO8xxR**: Check if reminder was given
- **COINSTRC**: Check level of impairment.  Base your answer only on observations made during Word Recognition task (Question #8)
- **COCOMPRE**: Check level of impairment
- **COWRDFND**: Check one response (see dictionary)
- **CONMCXLA**: 14a. Number of targets hit (range = 0-49)
- **CONMCXLB**: "14b. Number of errors","N","2","0..99","",""
- **CONMCXLC** 14c. Number of times reminded of task","N","2","0..99","",""

Data is not normalized

I think that the items are more less the same than for the ADNIGO,2,3 phases, but the total scores for each task are presented in the ADAS Sub-Scores and Total Scores dataset. 

In [11]:
ADAS_ADNI1 = pd.read_csv("Neuropsychological/ADAS_ADNI1.csv")
ADAS_ADNI1[ADAS_ADNI1['VISCODE']=='bl'].head()

,ID,RID,SITEID,VISCODE,USERDATE,USERDATE2,EXAMDATE,COT1LIST,COT2LIST,COT3LIST,COT1SCOR,COT2SCOR,COT3SCOR,COP1COMP,COCOMND,COP2COMP,COCONSTR,COP3COMP,COT4LIST,COT4TOTL,COP4COMP,CONAME,COP5COMP,COIDEA,COP6COMP,COORIEN,COP7COMP,CO8NURSE,CO8MAGAZ,CO8WIZRD,CO8WIZRR,CO8VAN,CO8VANR,CO8LEPRD,CO8LEPRR,CO8SALE,CO8SALER,CO8SEA,CO8SEAR,CO8TRAIN,CO8TRAIR,CO8COIN,CO8COINR,CO8SHIP,CO8SHIPR,CO8INST,CO8INSTR,CO8MAP,CO8MAPR,CO8AXE,CO8AXER,CO8BOARD,CO8BOARR,CO8CARRT,CO8CARRR,CO8MILK,CO8MILKR,CO8VOL,CO8VOLR,CO8FORST,CO8FORSR,CO8ANCHR,CO8ANCRR,CO8GEM,CO8GEMR,CO8CAT,CO8CATR,CO8FUND,CO8FUNDR,CO8EDGE,CO8EDGER,CO8CAKE,CO8CAKER,COP8COMP,COINSTRC,COCOMPRE,COWRDFND,COLANG,CONMCXLA,CONMCXLB,CONMCXLC,COP14CMP,update_stamp
0,4,2,107,bl,2005-09-09,NaN,2005-09-08,1:2:7:8:9,1:3:4:5:7:9:10,3:5:6:7:8:9:10,5,7,7,-4,1:2:3:4:5,-4,2:3:4:5,-4,2:3:7:8:10,5,-4,1:2:3:4:5:6:7:8:9:10:11:12:13:14:15:16:17,-4,1:2:3:4:5,-4,1:2:3:4:5:6:7:8,-4,0,1,1,-4,1,-4,1,-4,0,-4,0,-4,1,-4,0,-4,0,-4,1,-4,0,-4,0,-4,0,-4,0,-4,0,-4,0,-4,0,-4,0,-4,0,-4,0,-4,0,-4,1,-4,0,-4,-4,1,1,1,1,12,0,0,-4,2005-09-09 00:00:00.0
1,6,5,107,bl,2005-09-09,NaN,2005-09-07,1:2:3:7:9,1:3:5:7:8:9:10,2:3:4:5:7:9:10,5,7,7,-4,1,-4,2:3:4:5,-4,1:2:6:7:9:10,6,-4,1:2:3:4:5:6:7:8:9:10:11:12:13:14:15:16:17,-4,1:2:3:4:5,-4,1:2:3:4:5:6:7:8,-4,0,1,1,-4,1,-4,1,-4,0,-4,1,-4,1,-4,1,-4,0,-4,1,-4,0,-4,0,-4,0,-4,0,-4,0,-4,0,-4,0,-4,1,-4,1,-4,0,-4,1,-4,0,-4,0,-4,-4,1,1,1,1,16,0,0,-4,2005-09-09 00:00:00.0
2,8,3,107,bl,2005-09-13,NaN,2005-09-12,7:09:10,1:7:9:10,5:7:8:9:10,3,4,5,-4,1:2:3:4,-4,2:03:04,-4,7:09,2,-4,1:2:3:4:5:6:7:8:9:10:11:13:14:16:17,-4,1:2:3:4:5,-4,1:2:3:4,-4,0,1,0,-4,0,-4,1,-4,0,-4,1,-4,1,-4,0,-4,1,-4,0,-4,0,-4,0,-4,1,-4,0,-4,0,-4,0,-4,0,-4,1,-4,0,-4,0,-4,0,-4,0,-4,1,-4,-4,1,1,3,1,20,0,0,-4,2005-09-13 00:00:00.0
3,10,8,107,bl,2005-09-26,NaN,2005-09-19,1:2:4:5:7:8:9:10,1:2:3:4:5:6:7:8:9:10,1:2:3:4:5:6:8:9:10,8,10,9,-4,1:2:3:4:5,-4,2:03:04,-4,1:2:3:4:5:6:7:8,8,-4,1:2:3:4:5:6:7:8:9:10:11:12:13:14:15:16:17,-4,1:2:3:4:5,-4,1:2:3:4:5:6:7:8,-4,0,1,1,-4,1,-4,1,-4,0,-4,1,-4,0,-4,1,-4,0,-4,1,-4,0,-4,0,-4,1,-4,0,-4,0,-4,0,-4,0,-4,1,-4,0,-4,0,-4,0,-4,0,-4,0,-4,-4,1,1,1,1,23,0,0,-4,2005-09-26 00:00:00.0
4,12,7,10,bl,2005-10-06,NaN,2005-10-06,-4,7:8:9:10,4:8:9:10,0,4,4,-4,1:2:3:4,-4,2:04,-4,-4,0,-4,2:3:4:7:8:9:13:14,-4,1:2:3:4:5,-4,1:5:6:8,-4,1,1,1,-4,1,-4,1,-4,1,-4,1,-4,1,-4,1,-4,1,-4,1,-4,0,-4,0,-4,1,-4,1,-4,0,-4,0,-4,0,-4,0,-4,0,-4,0,-4,0,-4,0,-4,0,-4,-4,1,2,2,2,11,1,1,-4,2005-10-06 00:00:00.0


In [12]:
len(ADAS_ADNI1[ADAS_ADNI1['VISCODE']=='bl'])

819

In [78]:
ADAS_ADNI1['VISCODE'].unique()

array(['bl', 'm06', 'm24', 'm12', 'm48', 'm18', 'uns1', 'm36'],
      dtype=object)

### Alzheimer's Disease Assessment Scale (ADAS) [ADNIGO2,3] 

The ADAS is a brief cognitive test battery that assesses **learning and memory, language production, language comprehension, constructional praxis, ideational praxis, and orientation**. The ADAS is not a timed test and the
subject’s score does not depend upon how rapidly the test is completed.

The tasks are:

1. **Word Recall**: On this task, the subject is given three trials to learn a list of 10 words.
  - `WORDLIST`: type of list (e.g. English word list 1, English list 2, Spanish list...)
  - `Q1UNABLE`: patient was unable to complete task 
  - `Q1SCORE`: task 1 score (0..10).
  
2. **Commands**: This task is designed to assess receptive speech. The subject is asked to carry out five separate commands with 1 to 5 steps per command. `Q2SCORE`: task 2 score (0..5).

3. **Constructional Praxis**: This test assesses the subject’s ability to copy 4 geometric figures ranging from a very simple one (circle). `Q3SCORE`: task 3 score (0..5).
  
4. **Delayed Word Recall**: Delayed free recall of the word list should be attempted approximately 5 minutes after trial 3 of the Word. `Q4SCORE`: task 4 score (0..10).
  
5. **Naming Task**: In this task, the subject is asked to first name 12 randomly presented real objects that have high (Flower, Bed, Whistle, Pencil), medium (Rattle, Mask, Scissors, Comb), or low (Wallet, Harmonica, Stethoscope, Tongs) frequency values, and then to name the fingers on his/her dominant hand. `Q5SCORE`: task 5 score (0..5).

6. **Ideational Praxis**: his task is designed to determine whether the subject can perform a familiar but complex sequence of actions. For example, prepare a letter (envelope, address, stamp...). `Q6SCORE`: task 6 score (0..5).

7. **Orientation**: This task is designed to determine how well-oriented the subject is with regard to time and place. The components of orientation are: ➪ Person ➪ Year ➪ Day of the Week ➪ Season ➪ Month ➪ Time of Day ➪ Date ➪ Place. `Q7SCORE`: task 7 score (0..8).

8. **Word Recognition Task**: On this task, the subject is given one trial to learn a list of 12 words, presented one at a time, that are printed in block letters on white card. Then, the doctor is going to present them another set of words and the patient has to say which ones where in the previous list. `Q8SCORE`: task 8 (0..12).

9. **Remembering Test Instructions**: This item evaluates the subject’s ability to remember instructions and is based only on observations during the Word Recognition task. `Q9SCORE`: task 9 score (0..5).

10. **Comprehension**: This item evaluates the subject’s ability to understand speech. To rate this item the examiner should consider how well the subject was able to understand the examiner’s speech during the opening discussion and throughout the test session. `Q10SCORE`: question 10 score (0..5).

11. **Word Finding Difficulty**: This item rates impairment in expressive speech. The examiner rates the level of difficulty the subject had in finding the desired word in spontaneous speech during the interview and test session. `Q11SCORE`: question 11 score (0..5).

12. **Spoken Language Ability**: This item is a global rating of the quality of the subject’s speech and their ability to communicate verbally (i.e., clarity, difficulty in making oneself understood). `Q12SCORE`: question 12 score (0..5). 

13. **Number Cancellation**: identify some numbers among other numbers and cross them off. `Q13SCORE`: task 13 score (0..5).  

Other features that might be relevant:

- **TOTSCORE**: Total Score ADAS 11
- **TOTAL13**: Total Score ADAS 13

The original ADAS-Cog (Rosen et al. 1984) included **11 items** assessing cognitive function. The **modified ADAS-Cog 13-item scale** (Mohs et al. 1997) includes all original ADAS-Cog items with the addition of a number cancellation task and a delayed free recall task, for a total of 85 points. 

The tasks look the same than in ADAS [ADNI1], but I am not sure whether the rating system has changed. 

In [74]:
ADAS_ADNIGO23 = pd.read_csv("Neuropsychological/ADAS_ADNIGO23.csv")
ADAS_ADNIGO23[ADAS_ADNIGO23['VISCODE2']=='bl'].head()

,Phase,ID,RID,SITEID,VISCODE,VISCODE2,USERDATE,USERDATE2,WORDLIST,Q1UNABLE,Q1TR1,Q1TR2,Q1TR3,Q1TRIT,Q1TR2T,Q1TRT,Q1SCORE,TIMEEND,Q2UNABLE,Q2TASK,Q2SCORE,Q3UNABLE,Q3TASK1,Q3TASK2,Q3TASK3,Q3TASK4,Q3SCORE,Q4UNABLE,TIMEBEGAN,Q4TASK,Q4SCORE,Q5UNABLE,Q5TASK,Q5SCORE,Q5NAME1,Q5NAME2,Q5NAME3,Q5NAME4,Q5NAME5,Q5NAME6,Q5NAME7,Q5NAME8,Q5NAME9,Q5NAME10,Q5NAME11,Q5NAME12,Q5FINGER,Q5SCORE_CUE,Q6UNABLE,Q6TASK,Q6SCORE,Q7UNABLE,Q7TASK,Q7SCORE,Q8UNABLE,Q8WORD1,Q8WORD1R,Q8WORD2,Q8WORD2R,Q8WORD3,Q8WORD3R,Q8WORD4,Q8WORD4R,Q8WORD5,Q8WORD5R,Q8WORD6,Q8WORD6R,Q8WORD7,Q8WORD7R,Q8WORD8,Q8WORD8R,Q8WORD9,Q8WORD9R,Q8WORD10,Q8WORD10R,Q8WORD11,Q8WORD11R,Q8WORD12,Q8WORD12R,Q8WORD13,Q8WORD13R,Q8WORD14,Q8WORD14R,Q8WORD15,Q8WORD15R,Q8WORD16,Q8WORD16R,Q8WORD17,Q8WORD17R,Q8WORD18,Q8WORD18R,Q8WORD19,Q8WORD19R,Q8WORD20,Q8WORD20R,Q8WORD21,Q8WORD21R,Q8WORD22,Q8WORD22R,Q8WORD23,Q8WORD23R,Q8WORD24,Q8WORD24R,Q8SCORE,Q9TASK,Q9SCORE,Q10TASK,Q10SCORE,Q11TASK,Q11SCORE,Q12TASK,Q12SCORE,COMM,TOTSCORE,Q13UNABLE,Q13TASKA,Q13TASKB,Q13TASKC,Q13SCORE,TOTAL13,DATE,SOURCE,update_stamp
5,ADNIGO,22,2002,48,bl,bl,2010-07-06,2011-02-28,1.0,NaN,0:1:2:3:5:6:7,0:1:2:3:4:5:6:7:8:9,0:2:3:4:5:6:7:8:9,NaN,NaN,NaN,1.0,NaN,NaN,1:2:3:4:5,0.0,NaN,1.0,1.0,1.0,1.0,0.0,NaN,NaN,0:1:3:4:6:7:8:9,2.0,NaN,1:2:3:4:5:6:7:8:9:10:11:13:14:15:16:17,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1:2:3:4:5,0.0,NaN,1:2:3:4:5:6:7:8,0.0,NaN,0.0,-4.0,1.0,-4.0,1.0,-4.0,1.0,-4.0,1.0,-4.0,0.0,-4.0,1.0,-4.0,1.0,-4.0,0.0,-4.0,0.0,-4.0,1.0,-4.0,0.0,-4.0,0.0,-4.0,1.0,-4.0,0.0,-4.0,0.0,-4.0,0.0,-4.0,0.0,-4.0,1.0,-4.0,1.0,-4.0,0.0,-4.0,1.0,-4.0,0.0,-4.0,0.0,-4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,2.0,NaN,31.0,0.0,0.0,0.0,4.0,NaN,NaN,2012-11-16 09:32:21.0
6,ADNIGO,24,2007,12,bl,bl,2010-07-07,2011-02-28,1.0,NaN,1:4:7:8:9,0:3:4:6:7:8:9,0:2:3:4:5:6:7:8:9,NaN,NaN,NaN,3.0,NaN,NaN,1:2:3:4:5,0.0,NaN,1.0,1.0,1.0,2.0,1.0,NaN,NaN,0:1:2:4:5:6:7:8,2.0,NaN,1:2:3:4:5:6:7:8:9:10:11:13:14:15:16:17,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1:2:3:4:5,0.0,NaN,1:2:3:4:5:6:7:8,0.0,NaN,0.0,-4.0,1.0,-4.0,1.0,-4.0,1.0,-4.0,1.0,-4.0,0.0,-4.0,1.0,-4.0,1.0,-4.0,0.0,-4.0,0.0,-4.0,0.0,-4.0,0.0,-4.0,0.0,-4.0,1.0,-4.0,0.0,-4.0,0.0,-4.0,0.0,-4.0,0.0,-4.0,1.0,-4.0,1.0,-4.0,0.0,-4.0,0.0,-4.0,0.0,-4.0,0.0,-4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,7.0,NaN,23.0,0.0,0.0,0.0,9.0,NaN,NaN,2012-11-16 09:32:21.0
7,ADNIGO,26,2010,1,bl,bl,2010-07-09,2011-02-28,1.0,NaN,0:1:4:6:7:8:9,0:2:3:4:5:6:7:8:9,0:1:2:3:5:6:8:9,NaN,NaN,NaN,2.0,NaN,NaN,1:2:3:4:5,0.0,NaN,1.0,1.0,1.0,1.0,0.0,NaN,NaN,0:1:2:4:5:7:9,3.0,NaN,1:2:3:4:5:6:7:8:9:10:11:12:13:14:15:16:17,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1:2:3:4:5,0.0,NaN,1:2:3:4:5:6:7:8,0.0,NaN,0.0,-4.0,1.0,-4.0,1.0,-4.0,1.0,-4.0,1.0,-4.0,0.0,-4.0,1.0,-4.0,1.0,-4.0,1.0,-4.0,0.0,-4.0,1.0,-4.0,0.0,-4.0,0.0,-4.0,1.0,-4.0,0.0,-4.0,0.0,-4.0,0.0,-4.0,0.0,-4.0,1.0,-4.0,1.0,-4.0,0.0,-4.0,0.0,-4.0,0.0,-4.0,0.0,-4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,3.0,NaN,30.0,0.0,0.0,0.0,6.0,NaN,NaN,2012-11-16 09:32:21.0
10,ADNIGO,32,2011,48,bl,bl,2010-07-21,2011-02-28,1.0,NaN,0:1:2:3:6:7:9,0:1:2:3:4:5:9,0:1:2:3:4:5:8:9,NaN,NaN,NaN,3.0,NaN,NaN,1:2:3:4:5,0.0,NaN,1.0,1.0,1.0,2.0,1.0,NaN,NaN,0:1:2:3:4:5:7:9,2.0,NaN,1:2:3:4:5:6:7:8:9:10:11:12:13:14:15:16:17,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1:2:3:4:5,0.0,NaN,1:2:3:4:5:6:7:8,0.0,NaN,0.0,-4.0,1.0,-4.0,1.0,-4.0,1.0,-4.0,1.0,-4.0,0.0,-4.0,1.0,-4.0,1.0,-4.0,1.0,-4.0,0.0,-4.0,1.0,-4.0,0.0,-4.0,0.0,-4.0,1.0,-4.0,0.0,-4.0,0.0,-4.0,0.0,-4.0,0.0,-4.0,1.0,-4.0,1.0,-4.0,0.0,-4.0,1.0,-4.0,0.0,-4.0,0.0,-4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,4.0,NaN,18.0,0.0,0.0,1.0,7.0,NaN,NaN,2012-11-16 09:32:21.0
13,ADNIGO,38,2003,48,bl,bl,2010-07-23,2011-02-28,1.0,NaN,0:3:6:9,0:1:2:4:5:6:9,0:1:2:4:5:6:8:9,NaN,NaN,NaN,4.0,NaN,NaN,1:2:3:4:5,0.0,NaN,1.0,1.0,1.0,2.0,1.0,NaN,NaN,0:2:3:4:6:7:8:9,2.0,NaN,1:2:3:4:5:6:7:8:9:10:11:13:14:15:16:17,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1:2:3:4:5,0.0,NaN,1:2:3:4:5:6:7:8,0.

In [75]:
len(ADAS_ADNIGO23[ADAS_ADNIGO23['VISCODE2']=='bl'])

1606

In [77]:
ADAS_ADNIGO23['VISCODE2'].unique()

array(['m48', 'm36', 'bl', 'm60', 'm06', 'm12', 'm72', 'm24', 'm84',
       'm96', 'm108', 'm120', nan, 'm132', 'm144', 'm102', 'm30', 'm114',
       'm126', 'm186', 'm54', 'm138', 'm156', 'm180', 'm42', 'm18', 'm90',
       'm168', 'm174', 'm150', 'm162', 'm78', 'm66', 'm192', 'm198'],
      dtype=object)

### Clinical Dementia Rating Scale (CDR) [ADNI1,GO,2,3]

The CDR is a global rating scale for staging patients diagnosed with dementia (Table 16.6). The CDR evaluates cognitive, behavioral, and functional aspects of Alzheimer disease and other dementias. 

CDR is calculated on the basis of testing **six different cognitive and behavioral domains** such as memory, orientation, judgment and problem solving, community affairs, home and hobbies performance, and personal care. 

The CDR is based on a scale of 0–3: no dementia (CDR = 0), questionable dementia (CDR = 0.5), MCI (CDR = 1), moderate cognitive impairment (CDR = 2), and severe cognitive impairment (CDR = 3).

In [25]:
CDR = pd.read_csv("Neuropsychological/CDR.csv")
CDR.head()

,Phase,ID,RID,SITEID,VISCODE,VISCODE2,USERDATE,USERDATE2,EXAMDATE,CDSOURCE,CDVERSION,CDMEMORY,CDORIENT,CDJUDGE,CDCOMMUN,CDHOME,CDCARE,CDGLOBAL,CDRSB,CDSOB,CDDATE,update_stamp
0,ADNI1,8,2,107,sc,sc,2005-08-17,2009-10-05,2005-08-17,1.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,2009-10-05 00:00:00.0
1,ADNI1,10,3,107,sc,sc,2005-08-18,2009-10-05,2005-08-18,1.0,NaN,1.0,1.0,1.0,1.0,0.5,0.0,1.0,NaN,NaN,NaN,2009-10-05 00:00:00.0
2,ADNI1,12,4,10,sc,sc,2005-08-18,2009-10-05,2005-08-18,1.0,NaN,0.5,0.0,0.5,0.0,0.0,0.0,0.5,NaN,NaN,NaN,2009-10-05 00:00:00.0
3,ADNI1,14,5,107,sc,sc,2005-08-23,2009-10-05,2005-08-23,1.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,2009-10-05 00:00:00.0
4,ADNI1,16,7,10,sc,sc,2005-08-25,2009-10-05,2005-08-25,1.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,2009-10-05 00:00:00.0


In [76]:
len(CDR[CDR['VISCODE2']=='sc'])

3214

In [23]:
CDR['VISCODE'].unique()

array(['sc', 'f', 'm06', 'm12', 'm36', 'm18', 'uns1', 'm24', 'm48', 'm60',
       'm72', 'v01', 'v06', 'v11', 'v05', 'v21', 'v31', 'v41', 'v51',
       'init', 'y1', 'y2', 'y3', 'y4', 'y5'], dtype=object)

In [24]:
CDR['VISCODE2'].unique()

array(['sc', 'f', 'm06', 'm12', 'm36', 'm18', 'uns1', 'm24', 'm48', 'm60',
       'm72', 'm84', 'm96', 'm108', 'm120', 'm132', 'm66', 'm54', 'm78',
       'm126', 'm42', 'm138', 'm144', 'm90', 'm150', nan, 'm102', 'm156',
       'm162', 'm168', 'm30', 'm174', 'm114', 'm180', 'm186', 'm192',
       'm198'], dtype=object)

### Cognitive Change Index [ADNI2,3]

The Cognitive Change Index (CCI) is a tool used to assess the perception of cognitive decline in memory, executive function, and language domains from both self and informant perspectives. The present **20 item version** of the CCI was adapted from a larger item pool used in previous research on cognitive complaints in older adults by Saykin et al. with item selection based primarily on considerations regarding targeted content within and across the three included domains (**episodic memory, executive function, and language**) as well as analyses (unpublished data) on associations with neuroimaging, cognitive and outcome variables in an independent sample.

Participants and informants were asked to complete the CCI by responding to each item on a 1 to 5 Likert scale with higher scores indicating greater decline (1 = no change or normal ability, 2 = minimal change or slight/occasional problem, 3 = some change or mild problem, 4 = clearly noticeable change or moderate problem, 5 = much worse or severe problem). 

12 of the 20 CCI items focus on **memory performance** (e.g., “recalling information when I really try”, “remembering things that have happened recently”), 5 of the 20 items evaluate one’s **executive function** (e.g., “focusing on goals and carrying out a plan”), and 3 of the 20 items evaluate **language** (e.g., “understanding conversations”). 

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4833578/

In [27]:
CCI = pd.read_csv("Neuropsychological/CCI.csv")
CCI.head()

,Phase,ID,RID,SITEID,VISCODE,VISCODE2,USERDATE,USERDATE2,CCI1,CCI2,CCI3,CCI4,CCI5,CCI6,CCI7,CCI8,CCI9,CCI10,CCI11,CCI12,CCI13,CCI14,CCI15,CCI16,CCI17,CCI18,CCI19,CCI20,CCI12TOT,DATE,SOURCE,update_stamp
0,ADNI2,4,5072,19,v01,sc,2013-01-24,NaN,2.0,2.0,2.0,1.0,1.0,3.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,1.0,21.0,NaN,NaN,2013-05-16 19:16:06.0
1,ADNI2,6,5074,13,v01,sc,2013-01-28,NaN,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,1.0,3.0,2.0,2.0,2.0,3.0,3.0,2.0,1.0,2.0,2.0,2.0,23.0,NaN,NaN,2013-05-16 19:16:06.0
2,ADNI2,8,5082,27,v01,sc,2013-02-13,NaN,3.0,2.0,3.0,3.0,3.0,3.0,3.0,2.0,2.0,3.0,3.0,2.0,1.0,1.0,1.0,1.0,2.0,3.0,1.0,1.0,32.0,NaN,NaN,2013-05-16 19:16:06.0
3,ADNI2,10,5078,27,v01,sc,2013-02-14,NaN,4.0,4.0,3.0,3.0,3.0,4.0,4.0,4.0,3.0,4.0,3.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,4.0,2.0,42.0,NaN,NaN,2013-05-16 19:16:06.0
4,ADNI2,12,5079,19,v01,sc,2013-02-18,NaN,3.0,2.0,3.0,4.0,1.0,1.0,3.0,3.0,2.0,2.0,3.0,2.0,2.0,2.0,1.0,2.0,1.0,1.0,2.0,1.0,29.0,NaN,NaN,2013-05-16 19:16:06.0


In [92]:
len(CCI[CCI['VISCODE2']=='sc'])

1218

In [21]:
CCI['VISCODE'].unique()

array(['v01', 'sc'], dtype=object)

In [22]:
CCI['VISCODE2'].unique()

array(['sc'], dtype=object)

### Cogstate Battery Results [ADNI2,3] 

CogState is a computer administered test that assesses a number of cognitive skills including **reaction time, working memory, learning and attention**. Test scores are computer generated thus reducing the risk of human error in administration and scoring and allowing for more exact timing of response speed.

Each task in the battery is constructed using playing cards as stimuli with the test taker required to answer only “yes” or “no” on each trial in accord with a simple rule. The simple stimuli, rules and responses have been combined to generate cognitive paradigms that have been well-validated in neuropsychological and cognitive studies. These include measures of psychomotor function (**Detection task**), visual attention (**Identification task**), working memory (**One Back task**) and visual learning set within a pattern separation model (**One Card Learning task**).

**Source paper**: Maruff, P., Lim, Y.Y., Darby, D. et al. Clinical utility of the cogstate brief battery in identifying cognitive impairment in mild cognitive impairment and Alzheimer’s disease. BMC Psychol 1, 30 (2013). https://doi.org/10.1186/2050-7283-1-30

- `TestCode`: Code name for the test.

- `TestAttempt`: Attempt number. 

- `TestCompletionScore`: The completion score for the test [(TotalTrials/required correct responses) x 100; required correct responses is 35 for Detection, 30 for Identification, 80 for One Card Learning and 31 for One Back].

- `TestPerformanceScore`: Number The performance score for the test [reflects raw accuracy of performance; (TotalCorrect/TotalResponse) x 100]

- `ReactionTime`: Mean of log10 transformed reaction times of correct responses. 

- `RawReactionTime`: Milliseconds Mean of reaction times of correct responses.

- `RawAccuracy`: 4 Number Accuracy of performance (proportion of correct responses).


- `Psyattstdscr`: The score on the **Psychomotor Function/Attention composite**, incorporating ReactionTime as the outcome measure for the Detection and Identification tests, relative to age-matched normative data.

- `LearnWMStdScr`: The score on the **Learning/Working Memory composite**, incorporating Accuracy as the outcome measures for the One Card Learning and One Back tests, relative to age-matched normative data.

- `AltLearnWMStdScr`: The score on the Learning/Working Memory composite, incorporating Accuracy as the outcome measure for the One Card Learning test and ReactionTime as the outcome measure for the One Back test, relative to age-matched normative data.

Each **row** in the data extract represents **one test** in the Cogstate battery for a single subject at a single session. Thus, each patient identified by the RID has multiple records. 

More info in: https://adni.bitbucket.io/reference/cbb_results.html

There is also more info in the Cogstate PDF provided with the dataset. 


In [19]:
CBBR = pd.read_csv("Neuropsychological/ADNI_CBBRESULTS.csv")
CBBR.head()

,ProtocolID,RID,SessionID,EXAMDATE,TestTime,Visit,SessionAttempt,SessionDuration,SessionCompletionPass,SessionPerformancePass,SessionIntegrityPass,TestCode,TestAttempt,TestCompletionScore,TestPerformanceScore,TestCompletionPass,TestPerformancePass,TestIntegrityPass,TestDuration,ReactionTime,RawReactionTime,RTVariability,RawRTVariability,Accuracy,RawAccuracy,TotalCorrect,TotalCorrectExclPant,TotalErrors,LegalErrors,RuleBreakErrors,TotalAnticipatory,TotalPost,TotalMaxOut,TotalResponses,TotalTrials,StandardScoreZ,StandardScoreT,AltStandardScoreZ,AltStandardScoreT,Psyattstdscr,LearnWMStdScr,AltLearnWMStdScr,update_stamp
0,ADNI2,2,NaN,2015-12-08,16:58:11,NaN,3,976,Yes,NaN,NaN,OneBack,1,135.483871,73.913043,NaN,NaN,NaN,NaN,3.039397,NaN,0.262179,NaN,1.034735,NaN,34,NaN,12,NaN,NaN,NaN,NaN,NaN,46,42,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-06-19 15:01:35.0
1,ADNI2,2,NaN,2015-12-09,16:25:03,NaN,4,1114,Yes,NaN,NaN,Detection,1,100.000000,100.000000,NaN,NaN,NaN,NaN,2.770895,NaN,0.062567,NaN,1.570796,NaN,35,NaN,0,NaN,NaN,NaN,NaN,NaN,35,35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-06-19 15:01:35.0
2,ADNI2,2,NaN,2015-12-09,16:25:03,NaN,4,1114,Yes,NaN,NaN,Identification,1,100.000000,100.000000,NaN,NaN,NaN,NaN,2.922582,NaN,0.099267,NaN,1.570796,NaN,30,NaN,0,NaN,NaN,NaN,NaN,NaN,30,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-06-19 15:01:35.0
3,ADNI2,2,NaN,2015-12-09,16:25:03,NaN,4,1114,Yes,NaN,NaN,OneBack,1,106.451613,93.939394,NaN,NaN,NaN,NaN,3.199807,NaN,0.216261,NaN,1.322056,NaN,31,NaN,2,NaN,NaN,NaN,NaN,NaN,33,33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-06-19 15:01:35.0
4,ADNI2,2,NaN,2015-12-09,16:25:03,NaN,4,1114,Yes,NaN,NaN,OneCardLearning,2,100.000000,51.851852,NaN,NaN,NaN,NaN,3.208433,NaN,0.214661,NaN,0.803921,NaN,42,NaN,39,NaN,NaN,NaN,NaN,NaN,119,80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-06-19 15:01:35.0


In [20]:
CBBR['Visit'].unique()

array([nan, 'Remote', 'In Clinic', 'Unknown'], dtype=object)

In [125]:
len(CBBR['RID'].unique())

830

### Cogstate Brief Battery [ADNI3] 

The Cogstate Brief Battery provides a measure of four core cognitive domains: processing speed, attention, visual learning and working memory. 

- `DONE`: Was assessment/procedure done? "N","","0=No; 1=Yes","",""
- `NDREASON`: If No (assesment not done), reason not done: ; -4=NA
- `CBBPTCOMM`: Participant Comments
- `CBBCOMM`: Comments

There are no test scores ¿?

In [30]:
CBBCOMP = pd.read_csv("Neuropsychological/CBBCOMP.csv")
CBBCOMP.head()

,Phase,ID,RID,SITEID,VISCODE,VISCODE2,USERDATE,USERDATE2,DONE,NDREASON,CBBPTCOMM,CBBCOMM,update_stamp
0,ADNI2,8,5213,9,v21,NaN,2015-09-10,NaN,1,-4.0,-4,-4,2020-03-19 15:57:05.0
1,ADNI2,10,800,55,v41,NaN,2015-09-16,NaN,1,-4.0,-4,-4,2020-03-19 15:57:05.0
2,ADNI2,12,4128,9,v41,NaN,2015-09-18,NaN,1,-4.0,-4,-4,2020-03-19 15:57:05.0
3,ADNI2,14,5290,18,v21,NaN,2015-10-02,NaN,1,-4.0,Felt she responded too quickly to some of the ...,Easily administered.,2020-03-19 15:57:05.0
4,ADNI2,16,668,55,v41,NaN,2015-10-02,NaN,1,-4.0,-4,-4,2020-03-19 15:57:05.0


In [93]:
len(CBBCOMP[CBBCOMP['VISCODE2']=='bl'])

683

In [94]:
CBBCOMP['VISCODE2'].unique()

array([nan, 'm120', 'm72', 'm132', 'm66', 'm60', 'm54', 'm48', 'bl',
       'm78', 'm126', 'm42', 'm84', 'm138', 'm144', 'm12', 'm90', 'm150',
       'm96', 'm156', 'm102', 'm18', 'm24', 'm162', 'm108', 'm30', 'm168',
       'm36', 'm114', 'm174', 'm180', 'm186', 'm192', 'm198'],
      dtype=object)

###  Embic Corporation – Digital Cognitive Biomarkers 

To generate DCBs, an HBCP (hierarchical Bayesian cognitive processing) model is applied to item response data from commonly used **wordlist memory (WLM)** tests (e.g., ADAS-Cog, AVLT, CVLT, EVLT, and CERAD) with two or more immediate learning
and recall tasks and at least one delayed recall task. The model generates seven base
DCBs, each representing the **probability of information processed** through different encoding
(N1, N2, N3, or N4) or retrieval (R1, R2, or R3) pathways, to and from three distinct storage states
(pre-task, transient, or durable storage states).

In contrast to purely observed behaviors (e.g., the number of words recalled), this generalized probability of
recall, which includes the combination of encoding and retrieval processes that result in successful
recall across WLM test tasks, is constructed from base DCBs subsequent to modeling.

I guess that these means that raw scores are normalized somehow ¿?

In [32]:
EMBICDCB = pd.read_csv("Neuropsychological/ADNI_EMBICDCB.csv")
EMBICDCB.head()

,RID,VISCODE2,N1,N2,N3,N4,R1,R2,R3,M1,M2,M3,update_stamp
0,2,bl,0.380379,0.525858,0.608526,0.233617,0.312128,0.855483,0.630490,0.814638,0.847062,0.552671,2022-08-25 16:48:31.0
1,2,m06,0.355836,0.451357,0.405075,0.259381,0.282130,0.821923,0.628997,0.739732,0.752799,0.495499,2022-08-25 16:48:31.0
2,2,m36,0.456460,0.326185,0.530069,0.342751,0.417837,0.822434,0.567525,0.781423,0.851242,0.501393,2022-08-25 16:48:31.0
3,2,m60,0.356400,0.525238,0.342911,0.204434,0.326040,0.828272,0.659133,0.765615,0.733223,0.503721,2022-08-25 16:48:31.0
4,2,m72,0.328507,0.522139,0.489105,0.259510,0.295456,0.907406,0.613058,0.818876,0.795315,0.498164,2022-08-25 16:48:31.0


In [33]:
len(EMBICDCB[EMBICDCB['VISCODE2']=='bl'])

2258

In [95]:
EMBICDCB['VISCODE2'].unique()

array(['bl', 'm06', 'm36', 'm60', 'm72', 'm84', 'm96', 'm120', 'm132',
       'm144', 'm12', 'm24', 'm18', 'm150', 'm162', 'm108', 'm48', 'm156',
       'm168', 'm180', 'm138', 'm174', 'm126', 'm102', 'm78', 'm90',
       'm114', 'm66', 'm54', 'm42', 'm30'], dtype=object)

### Everyday Cognition - Participant Self Report [ADNIGO,2,3] 

The Everyday Cognition Scale (ECog) is a **39 item questionnaire** used to measure a patient's cognitive function and aid in the detection and diagnosis of dementia and other neurodegenerative diseases that are associated with aging. 

- **Memory**: 8 questions
- **Language**: 9 questions
- **Visual-spatial and perceptual abilities**: 8 questions
- **Executive functioning: planning**: 5 questions
- **Executive functioning: organization**: 6
- **Executive functioning: divided attention**: 4


The **final score** is an average of the individual scores for each item answered out of the 39 items in the questionnaire and ranges from 1 to 4. If an item is not answered or "I don't know" is given as the answer, a value of null is assigned for the purpose of calculating the final score. 

There are two versions of the ECog: one that the patient completes (**Participant Self Report**) and one that the patient's partner completes (**Study Partner Report**).

More info in: https://loinc.org/89133-3/

**Source paper**: Farias S Tomaszewski, Mungas D, Reed B, Cahn-Weiner D, Jagust W, Baynes K, et al. The measurement of everyday cognition (ECog): Scale development and psychometric properties. Neuropsychology. 2008;22:531–544.

In [34]:
ECOGSP = pd.read_csv("Neuropsychological/ECOGSP.csv")
ECOGSP.head()

,PHASE,ID,RID,SITEID,VISCODE,VISCODE2,USERDATE,USERDATE2,MEMORY1,MEMORY2,MEMORY3,MEMORY4,MEMORY5,MEMORY6,MEMORY7,MEMORY8,LANG1,LANG2,LANG3,LANG4,LANG5,LANG6,LANG7,LANG8,LANG9,VISSPAT1,VISSPAT2,VISSPAT3,VISSPAT4,VISSPAT5,VISSPAT6,VISSPAT7,VISSPAT8,PLAN1,PLAN2,PLAN3,PLAN4,PLAN5,ORGAN1,ORGAN2,ORGAN3,ORGAN4,ORGAN5,ORGAN6,DIVATT1,DIVATT2,DIVATT3,DIVATT4,SOURCE,EcogSPDivatt,EcogSPLang,EcogSPMem,EcogSPOrgan,EcogSPPlan,EcogSPVisspat,EcogSPTotal,COLDATE,update_stamp
0,ADNIGO,4,1326,12,m36,m36,2010-05-27,2011-08-04,3.0,3.0,4.0,4.0,2.0,4.0,4.0,2.0,2.0,2.0,2.0,1.0,4.0,2.0,1.0,2.0,2.0,3.0,3.0,2.0,2.0,NaN,1.0,2.0,1.0,3.0,2.0,3.0,1.0,2.0,2.0,3.0,3.0,2.0,2.0,4.0,3.0,3.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012-11-16 09:40:21.0
1,ADNIGO,6,292,24,m48,m48,2010-05-27,NaN,3.0,4.0,4.0,3.0,4.0,4.0,4.0,4.0,1.0,2.0,1.0,1.0,3.0,1.0,1.0,2.0,3.0,4.0,4.0,4.0,3.0,2.0,2.0,2.0,1.0,3.0,2.0,4.0,2.0,3.0,2.0,9.0,4.0,3.0,3.0,4.0,3.0,3.0,3.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012-11-16 09:40:21.0
2,ADNIGO,8,376,17,m48,m48,2010-05-28,NaN,3.0,3.0,3.0,3.0,2.0,3.0,2.0,3.0,2.0,3.0,4.0,4.0,2.0,2.0,2.0,2.0,3.0,2.0,2.0,2.0,2.0,2.0,3.0,2.0,2.0,3.0,3.0,3.0,3.0,3.0,2.0,3.0,3.0,3.0,3.0,2.0,4.0,3.0,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012-11-16 09:40:21.0
3,ADNIGO,10,1117,12,m36,m36,2010-06-27,2011-11-08,3.0,2.0,3.0,2.0,3.0,4.0,3.0,4.0,1.0,2.0,2.0,1.0,2.0,1.0,9.0,2.0,1.0,9.0,9.0,1.0,9.0,NaN,1.0,1.0,1.0,1.0,1.0,2.0,3.0,9.0,1.0,3.0,9.0,9.0,2.0,9.0,2.0,1.0,2.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012-11-16 09:40:21.0
4,ADNIGO,12,2007,12,bl,bl,2010-07-06,NaN,3.0,1.0,1.0,3.0,3.0,1.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,3.0,3.0,2.0,1.0,3.0,1.0,3.0,2.0,1.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012-11-16 09:40:21.0


In [96]:
len(ECOGSP[ECOGSP['VISCODE2']=='bl'])

1590

In [97]:
ECOGSP['VISCODE2'].unique()

array(['m36', 'm48', 'bl', 'm60', 'm06', 'm12', 'm72', 'm24', 'm84',
       'm96', 'm108', 'm120', 'm132', 'm66', 'm138', 'm78', 'm144', 'm54',
       'm126', 'm90', 'm150', nan, 'm156', 'm18', 'm102', 'm42', 'm162',
       'm30', 'm168', 'm174', 'm114', 'm180', 'm186', 'm192', 'm198'],
      dtype=object)

### Everyday Cognition - Study Partner Report [ADNIGO,2,3] 

In [36]:
ECOGPT = pd.read_csv("Neuropsychological/ECOGPT.csv")
ECOGPT.head()

,Phase,ID,RID,SITEID,VISCODE,VISCODE2,USERDATE,USERDATE2,CONCERN,MEMORY1,MEMORY2,MEMORY3,MEMORY4,MEMORY5,MEMORY6,MEMORY7,MEMORY8,LANG1,LANG2,LANG3,LANG4,LANG5,LANG6,LANG7,LANG8,LANG9,VISSPAT1,VISSPAT2,VISSPAT3,VISSPAT4,VISSPAT5,VISSPAT6,VISSPAT7,VISSPAT8,PLAN1,PLAN2,PLAN3,PLAN4,PLAN5,ORGAN1,ORGAN2,ORGAN3,ORGAN4,ORGAN5,ORGAN6,DIVATT1,DIVATT2,DIVATT3,DIVATT4,STAFFASST,VALIDITY,SOURCE,EcogPtDivatt,EcogPtLang,EcogPtMem,EcogPtOrgan,EcogPtPlan,EcogPtVisspat,EcogPtTotal,COLDATE,update_stamp
0,ADNIGO,6,1326,12,m36,m36,2010-05-27,2011-08-04,NaN,4.0,3.0,4.0,4.0,3.0,4.0,4.0,4.0,3.0,3.0,3.0,3.0,3.0,3.0,1.0,4.0,4.0,3.0,2.0,2.0,2.0,NaN,1.0,1.0,1.0,3.0,2.0,3.0,3.0,4.0,3.0,3.0,3.0,3.0,3.0,3.0,2.0,3.0,3.0,3.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012-11-16 09:40:16.0
1,ADNIGO,8,292,24,m48,m48,2010-05-27,NaN,NaN,2.0,2.0,3.0,2.0,2.0,3.0,3.0,3.0,3.0,2.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,9.0,9.0,2.0,2.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012-11-16 09:40:16.0
2,ADNIGO,10,376,17,m48,m48,2010-05-28,NaN,NaN,3.0,2.0,3.0,3.0,2.0,4.0,4.0,9.0,2.0,3.0,3.0,3.0,4.0,2.0,3.0,3.0,2.0,1.0,1.0,3.0,3.0,4.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,3.0,9.0,9.0,2.0,2.0,3.0,3.0,4.0,3.0,4.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012-11-16 09:40:16.0
3,ADNIGO,12,618,21,m48,m48,2010-06-28,NaN,NaN,1.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012-11-16 09:40:16.0
4,ADNIGO,14,1117,12,m36,m36,2010-07-02,2011-11-09,NaN,2.0,1.0,1.0,2.0,2.0,2.0,2.0,3.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,1.0,2.0,NaN,1.0,1.0,1.0,9.0,1.0,1.0,1.0,1.0,1.0,9.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012-11-16 09:40:16.0


In [98]:
len(ECOGPT[ECOGPT['VISCODE2']=='bl'])

1605

In [99]:
ECOGPT['VISCODE2'].unique()

array(['m36', 'm48', 'bl', 'm60', 'm06', 'm12', 'm72', 'm24', 'm84',
       'm96', 'm108', 'm120', 'm132', 'm66', 'm78', 'm138', 'm144',
       'm126', 'm90', 'm54', 'm42', 'm150', nan, 'm102', 'm156', 'm18',
       'm162', 'm168', 'm30', 'm174', 'm114', 'm180', 'm186', 'm192',
       'm198'], dtype=object)

###  Financial Capacity Instrument Short Form (FCI-SF) [ADNI3] 

The Financial Capacity Instrument-Short Form (FCI-SF) is a performance-based **measure of everyday financial skills** that takes 15 min to administer. This short form evaluates coin/currency calculation, financial conceptual knowledge, use of a checkbook/register, and use of a bank statement. 

Items are scored and summed to calculate four component performance scores and a total score in addition to timing indexes of four tasks:

1. Single/Checkbook Register Score (`SNGLCHKSCORE`): Items 7-16
2. Complex Checkbook/Register Score (`COMPLXCHKSCORE`): Items 17-30
3. Combined Checkbook/Register Score (`CHKCOMPOSSCORE`): Items 7-16 + 17-30
4. Bank Statement Management Score (`BANKSTATSCORE`): Items 31-37

Other relevant parameters:

- Mental capacity score (`MC_SCORE`): Items 1-2
- Financial Conceptual Knowledge Score (`FC_SCORE`): Items 3-6
- TOTAL Composite Time (`TOTCOMPTIME`)
- TOTAL FCI-SF SCORE (`FCISCORE`)

The other features are the questions asked in the form. 

**Source paper**: Marson DC, Sawrie SM, Snyder S, et al. Assessing financial capacity in patients with Alzheimer disease: a conceptual model and prototype instrument. Arch Neurol. 2000;57:877–884. https://doi.org/10.1001/archneur.57.6.877

In [38]:
FCI = pd.read_csv("Neuropsychological/FCI.csv")
FCI.head()

,Phase,ID,RID,SITEID,VISCODE,VISCODE2,USERDATE,USERDATE2,MC_SCORE,FC_SCORE,SNGLCHKSCORE,COMPLXCHKSCORE,CHKCOMPOSTIME,CHKCOMPOSSCORE,BANKSTATSCORE,TOTCOMPTIME,FCISCORE,DONE,NDREASON,NICKELS,QUARTERS,BUDGET,EXPMEDINS,INSURANCE,MEDINSTIME,EXPINCTAX,TAXCREDA,QUES5TIME,TAXCREDB,EXPCHECKREG,PAYEE,SNGLCHK7,SNGLCHK8,SNGLCHK9,SNGLCHK10,SNGLCHK11,SNGLCHK12,SNGLCHK13,SNGLCHK14,SNGLCHK15,SNGLCHK16,SNGLCHKTIME,COMPLXCHK17,COMPLXCHK18,COMPLXCHK19,COMPLXCHK20,COMPLXCHK21,COMPLXCHK22,COMPLXCHK23,COMPLXCHK24,COMPLXCHK25,COMPLXCHK26,COMPLXCHK27,COMPLXCHK28,COMPLXCHK29,COMPLXCHK30,COMPLXCHKTIME,BANKSTATA,BANKSTATB,EXPBANKSTATE,BANKSTAT32,BANKSTAT33,BANKSTAT34,BANKSTAT35,BANKSTAT36,BANKSTAT37,update_stamp
0,ADNI3,297,74,20,init,m132,2017-01-17,2017-01-17,4.0,6.0,16.0,24.0,238.0,40.0,12.0,243.0,62.0,1,NaN,2.0,2.0,0.0,1.0,2.0,2.0,1.0,2.0,3.0,NaN,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,0.0,0.0,2.0,98.0,2.0,2.0,2.0,2.0,0.0,2.0,2.0,2.0,2.0,2.0,2.0,0.0,2.0,2.0,140.0,2.0,NaN,1.0,2.0,2.0,0.0,2.0,2.0,2.0,2017-09-19 09:09:06.0
1,ADNI3,942,1078,27,init,m120,2017-02-23,2017-02-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-08-02 07:59:36.0
2,ADNI3,245,2183,20,init,m72,2017-01-17,2017-01-17,4.0,5.0,14.0,16.0,410.0,30.0,9.0,436.0,48.0,1,NaN,2.0,2.0,1.0,1.0,0.0,17.0,1.0,2.0,9.0,NaN,1.0,2.0,2.0,2.0,2.0,0.0,2.0,0.0,2.0,0.0,2.0,2.0,110.0,2.0,0.0,2.0,2.0,2.0,2.0,0.0,2.0,2.0,2.0,99.0,99.0,99.0,99.0,300.0,0.0,1.0,0.0,2.0,2.0,0.0,2.0,0.0,2.0,2017-09-19 09:09:06.0
3,ADNI3,425,2219,20,init,m72,2017-01-24,2017-01-24,4.0,8.0,18.0,24.0,365.0,42.0,9.0,400.0,63.0,1,NaN,2.0,2.0,2.0,1.0,2.0,30.0,1.0,2.0,5.0,NaN,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,0.0,2.0,2.0,112.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,0.0,2.0,2.0,2.0,0.0,2.0,253.0,0.0,1.0,1.0,99.0,2.0,0.0,2.0,2.0,2.0,2017-09-19 09:09:06.0
4,ADNI3,229,2245,20,init,m72,2017-01-06,2017-01-06,4.0,8.0,20.0,28.0,339.0,48.0,14.0,404.0,74.0,1,NaN,2.0,2.0,2.0,1.0,2.0,54.0,1.0,2.0,11.0,NaN,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,106.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,233.0,2.0,NaN,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2017-09-19 09:09:06.0


In [101]:
len(FCI[FCI['VISCODE2']=='bl'])

687

In [102]:
FCI['VISCODE2'].unique()

array(['m132', 'm120', 'm72', 'm66', 'm60', 'm54', 'm48', 'bl', 'm78',
       'm126', 'm42', nan, 'm138', 'm84', 'm144', 'm12', 'm90', 'm150',
       'm96', 'm102', 'm18', 'm156', 'm24', 'm162', 'm108', 'm30', 'm168',
       'm36', 'm114', 'm174', 'm180', 'm186', 'm192', 'm198'],
      dtype=object)

### Functional Activities Questionnaire (FAQ) [ADNI1,GO,2,3] 

The Functional Activities Questionnaire (FAQ) serves a screening tool for evaluating **ADI’s independence**. In order to take the FAQ, a lay informant (i.e. spouse, friend, or relative) must administer it. The participant most respond to **10-items pertaining to daily tasks needed when living independently**. 

A scale ranging from independence (0) to dependence (3) is used for responses. Approximately 10 minutes is required for completion.

- `FAQSOURCE`: Information Source (visit, telephone call...)
- `FAQFINAN`: Writing checks, paying bills, balancing checkbook
- `FAQFORM`: Assembling tax records, business affairs, or papers
- `FAQSHOP`: Shopping alone for clothes, household necessities, or groceries
- `FAQGAME`: Playing a game of skill, working on a hobby
- `FAQBEVG`: Heating water, making a cup of coffee, turning off stove after use
- `FAQMEAL`: Preparing a balanced meal
- `FAQEVENT`: Keeping track of current events
- `FAQTV`: Paying attention to, understanding, discussing TV, book, magazine
- `FAQREM`: Remembering appointments, family occasions, holidays, medications
- `FAQTRAVL`: Traveling out of neighborhood, driving, arranging to take buses
- `FAQTOTAL`: Total score. It ranges from 0 to 30. 

In [41]:
FAQ = pd.read_csv("Neuropsychological/FAQ.csv")
FAQ.head()

,Phase,ID,RID,SITEID,VISCODE,VISCODE2,USERDATE,USERDATE2,EXAMDATE,FAQSOURCE,FAQFINAN,FAQFORM,FAQSHOP,FAQGAME,FAQBEVG,FAQMEAL,FAQEVENT,FAQTV,FAQREM,FAQTRAVL,FAQTOTAL,SPID,FAQDATE,update_stamp
0,ADNI1,4,2,107,bl,bl,2005-09-08,NaN,2005-09-08,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,2005-09-08 00:00:00.0
1,ADNI1,6,5,107,bl,bl,2005-09-09,NaN,2005-09-07,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,2005-09-09 00:00:00.0
2,ADNI1,8,3,107,bl,bl,2005-09-13,NaN,2005-09-12,1.0,2.0,3.0,2.0,3.0,1.0,1.0,4.0,3.0,3.0,4.0,10.0,NaN,NaN,2005-09-13 00:00:00.0
3,ADNI1,10,8,107,bl,bl,2005-09-26,NaN,2005-09-19,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,2005-09-26 00:00:00.0
4,ADNI1,12,7,10,bl,bl,2005-10-06,NaN,2005-10-06,1.0,5.0,5.0,4.0,2.0,0.0,3.0,4.0,3.0,5.0,3.0,17.0,NaN,NaN,2005-10-06 00:00:00.0


In [103]:
len(FAQ[FAQ['VISCODE2']=='bl'])

2417

In [104]:
FAQ['VISCODE2'].unique()

array(['bl', 'm06', 'm12', 'm18', 'uns1', 'm24', 'm36', 'm48', 'm60',
       'm72', 'm84', 'm96', 'm108', 'm120', nan, 'm132', 'm66', 'm54',
       'm78', 'm126', 'm42', 'm138', 'm144', 'm90', 'm150', 'm102',
       'm156', 'm162', 'm168', 'm30', 'm174', 'm114', 'm180', 'm186',
       'm192', 'm198'], dtype=object)

### Geriatric Depression Scale (GDS) [ADNI1,GO,2,3] 

The GDS Short Form is a brief, **15-item questionnaire** in which participants are asked to respond by answering yes (1.0) or no (0.0) in reference to how they felt over the past week. Of the 15 items, 10 indicated the presence of depression when answered positively, while the rest (question numbers 1, 5, 7, 11, 13) indicated depression when answered negatively. 

1. Are you basically **satisfied** with your life?
2. Have you **dropped** many of your activities and interests?
3. Do you feel that your life is **empty**?
4. Do you often get **bored**?
5. Are you in good **spirits** most of the time?
6. Are you **afraid** that something bad is going to happen to you?
7. Do you feel **happy** most of the time?
8. Do you often feel **helpless**?
9.  Do you prefer to stay at **home**, rather than going out and doing new things?
10. Do you feel you have more problems with **memory** than most?
11. Do you think its wonderful to be **alive** now?
12. Do you feel pretty **worthless** the way you are now?
13. Do you feel full of **energy**?
14. Do you feel that your situation is **hopeless**?
15. Do you think that most people are **better** off than you are?

Scores of 0-4 are considered normal, depending on age, education, and complaints; 5-8 indicate mild depression; 9-11 indicate moderate depression; and 12-15 indicate severe depression.

More info in: https://wwwoundcare.ca/Uploads/ContentDocuments/Geriatric%20Depression%20Scale.pdf

In [43]:
GDSCALE = pd.read_csv("Neuropsychological/GDSCALE.csv")
GDSCALE.head()

,Phase,ID,RID,SITEID,VISCODE,VISCODE2,USERDATE,USERDATE2,EXAMDATE,GDSOURCE,GDUNABL,GDUNABSP,GDSATIS,GDDROP,GDEMPTY,GDBORED,GDSPIRIT,GDAFRAID,GDHAPPY,GDHELP,GDHOME,GDMEMORY,GDALIVE,GDWORTH,GDENERGY,GDHOPE,GDBETTER,GDTOTAL,GDDATE,update_stamp
0,ADNI1,6,2,107,sc,sc,2005-08-17,NaN,2005-08-17,1.0,-4.0,-4,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,NaN,2005-08-17 00:00:00.0
1,ADNI1,8,3,107,sc,sc,2005-08-18,NaN,2005-08-18,1.0,-4.0,-4,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,NaN,2005-08-18 00:00:00.0
2,ADNI1,10,4,10,sc,sc,2005-08-18,NaN,2005-08-18,1.0,-4.0,-4,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,3.0,NaN,2005-08-18 00:00:00.0
3,ADNI1,12,5,107,sc,sc,2005-08-23,NaN,2005-08-23,1.0,-4.0,-4,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,NaN,2005-08-23 00:00:00.0
4,ADNI1,14,7,10,sc,sc,2005-08-25,NaN,2005-08-25,1.0,-4.0,-4,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,NaN,2005-08-25 00:00:00.0


In [106]:
len(GDSCALE[GDSCALE['VISCODE']=='sc'])

2179

In [48]:
GDSCALE['VISCODE'].unique()

array(['sc', 'f', 'm12', 'uns1', 'm24', 'm36', 'm48', 'm72', 'm06', 'm60',
       'v01', 'v06', 'v11', 'v05', 'v21', 'v31', 'v41', 'v51', 'init',
       'y1', 'y2', 'y3', 'y4', 'y5'], dtype=object)

### Item Level Data (ADAS-Cog, ANART, MMSE, etc) [ADNI1] 

Multiple items are often used for cognitive diagnosis. When respondents choose to answer some but not all items, **item-level missing data occur**. Respondents may refuse to answer items that they deem too difficult, quit the test early because it is too long, or just skip items because of carelessness. Missing data lead to loss of information and may result in biased conclusions. 

This table give some insight of which items from different test batteries were answered by the patients and, in some cases, the start and end time.

**Source paper**: Shan N, Wang X. Cognitive Diagnosis Modeling Incorporating Item-Level Missing Data Mechanism. Front Psychol. 2020 Nov 30;11:564707. doi: 10.3389/fpsyg.2020.564707

In [51]:
ITEM = pd.read_csv("Neuropsychological/ITEM.csv")
ITEM.head()

,RID,VISCODE,ADAS_QuestionnaireNotAttempted,ADAS_Q1_T1_WL1_W1_butter,ADAS_Q1_T1_WL1_W2_arm,ADAS_Q1_T1_WL1_W3_shore,ADAS_Q1_T1_WL1_W4_letter,ADAS_Q1_T1_WL1_W5_queen,ADAS_Q1_T1_WL1_W6_cabin,ADAS_Q1_T1_WL1_W7_pole,ADAS_Q1_T1_WL1_W8_ticket,ADAS_Q1_T1_WL1_W9_grass,ADAS_Q1_T1_WL1_W10_engine,ADAS_Q1_T2_WL1_W1_pole,ADAS_Q1_T2_WL1_W2_letter,ADAS_Q1_T2_WL1_W3_butter,ADAS_Q1_T2_WL1_W4_queen,ADAS_Q1_T2_WL1_W5_arm,ADAS_Q1_T2_WL1_W6_shore,ADAS_Q1_T2_WL1_W7_grass,ADAS_Q1_T2_WL1_W8_cabin,ADAS_Q1_T2_WL1_W9_ticket,ADAS_Q1_T2_WL1_W10_engine,ADAS_Q1_T3_WL1_W1_shore,ADAS_Q1_T3_WL1_W2_letter,ADAS_Q1_T3_WL1_W3_arm,ADAS_Q1_T3_WL1_W4_cabin,ADAS_Q1_T3_WL1_W5_pole,ADAS_Q1_T3_WL1_W6_ticket,ADAS_Q1_T3_WL1_W7_engine,ADAS_Q1_T3_WL1_W8_grass,ADAS_Q1_T3_WL1_W9_butter,ADAS_Q1_T3_WL1_W10_queen,ADAS_Q1_T1_WL2_W1_bottle,ADAS_Q1_T1_WL2_W2_potato,ADAS_Q1_T1_WL2_W3_girl,ADAS_Q1_T1_WL2_W4_temple,ADAS_Q1_T1_WL2_W5_star,ADAS_Q1_T1_WL2_W6_animal,ADAS_Q1_T1_WL2_W7_forest,ADAS_Q1_T1_WL2_W8_lake,ADAS_Q1_T1_WL2_W9_clock,ADAS_Q1_T1_WL2_W10_office,ADAS_Q1_T2_WL2_W1_forest,ADAS_Q1_T2_WL2_W2_temple,ADAS_Q1_T2_WL2_W3_bottle,ADAS_Q1_T2_WL2_W4_star,ADAS_Q1_T2_WL2_W5_potato,ADAS_Q1_T2_WL2_W6_girl,ADAS_Q1_T2_WL2_W7_clock,ADAS_Q1_T2_WL2_W8_animal,ADAS_Q1_T2_WL2_W9_lake,ADAS_Q1_T2_WL2_W10_office,ADAS_Q1_T3_WL2_W1_girl,ADAS_Q1_T3_WL2_W2_temple,ADAS_Q1_T3_WL2_W3_potato,ADAS_Q1_T3_WL2_W4_animal,ADAS_Q1_T3_WL2_W5_forest,ADAS_Q1_T3_WL2_W6_lake,ADAS_Q1_T3_WL2_W7_office,ADAS_Q1_T3_WL2_W8_clock,ADAS_Q1_T3_WL2_W9_bottle,ADAS_Q1_T3_WL2_W10_star,ADAS_Q1_T1_WL3_W1_coast,ADAS_Q1_T1_WL3_W2_doll,ADAS_Q1_T1_WL3_W3_lip,ADAS_Q1_T1_WL3_W4_chair,ADAS_Q1_T1_WL3_W5_student,ADAS_Q1_T1_WL3_W6_apple,ADAS_Q1_T1_WL3_W7_horse,ADAS_Q1_T1_WL3_W8_pipe,ADAS_Q1_T1_WL3_W9_valley,ADAS_Q1_T1_WL3_W10_rock,ADAS_Q1_T2_WL3_W1_horse,ADAS_Q1_T2_WL3_W2_chair,ADAS_Q1_T2_WL3_W3_coast,ADAS_Q1_T2_WL3_W4_student,ADAS_Q1_T2_WL3_W5_doll,ADAS_Q1_T2_WL3_W6_lip,ADAS_Q1_T2_WL3_W7_valley,ADAS_Q1_T2_WL3_W8_apple,ADAS_Q1_T2_WL3_W9_pipe,ADAS_Q1_T2_WL3_W10_rock,ADAS_Q1_T3_WL3_W1_lip,ADAS_Q1_T3_WL3_W2_chair,ADAS_Q1_T3_WL3_W3_doll,ADAS_Q1_T3_WL3_W4_apple,ADAS_Q1_T3_WL3_W5_horse,ADAS_Q1_T3_WL3_W6_pipe,ADAS_Q1_T3_WL3_W7_rock,ADAS_Q1_T3_WL3_W8_valley,ADAS_Q1_T3_WL3_W9_coast,ADAS_Q1_T3_WL3_W10_student,ADAS_Q1_TimeEnded,ADAS_Q2a,ADAS_Q2b,ADAS_Q2c,ADAS_Q2d,ADAS_Q2e,ADAS_Q3a,ADAS_Q3b,ADAS_Q3c,ADAS_Q3d,ADAS_Q4_TimeBegan,ADAS_Q4_WL1_W1_butter,ADAS_Q4_WL1_W2_arm,ADAS_Q4_WL1_W3_shore,ADAS_Q4_WL1_W4_letter,ADAS_Q4_WL1_W5_queen,ADAS_Q4_WL1_W6_cabin,ADAS_Q4_WL1_W7_pole,ADAS_Q4_WL1_W8_ticket,ADAS_Q4_WL1_W9_grass,ADAS_Q4_WL1_W10_engine,ADAS_Q4_WL2_W1_bottle,ADAS_Q4_WL2_W2_potato,ADAS_Q4_WL2_W3_girl,ADAS_Q4_WL2_W4_temple,ADAS_Q4_WL2_W5_star,ADAS_Q4_WL2_W6_animal,ADAS_Q4_WL2_W7_forest,ADAS_Q4_WL2_W8_lake,ADAS_Q4_WL2_W9_clock,ADAS_Q4_WL2_W10_office,ADAS_Q4_WL3_W1_coast,ADAS_Q4_WL3_W2_doll,ADAS_Q4_WL3_W3_lip,ADAS_Q4_WL3_W4_chair,ADAS_Q4_WL3_W5_student,ADAS_Q4_WL3_W6_apple,ADAS_Q4_WL3_W7_horse,ADAS_Q4_WL3_W8_pipe,ADAS_Q4_WL3_W9_valley,ADAS_Q4_WL3_W10_rock,ADAS_Q5a_Flower,ADAS_Q5a_Bed,ADAS_Q5a_Whistle,ADAS_Q5a_Pencil,ADAS_Q5a_Rattle,ADAS_Q5a_Mask,ADAS_Q5a_Scissors,ADAS_Q5a_Comb,ADAS_Q5a_Wallet,ADAS_Q5a_Harmonica,ADAS_Q5a_Stethoscope,ADAS_Q5a_Tongs,ADAS_Q5b_Thumb,ADAS_Q5b_Middle,ADAS_Q5b_Ring,ADAS_Q5b_Index,ADAS_Q5b_Pinky,ADAS_Q6a,ADAS_Q6b,ADAS_Q6c,ADAS_Q6d,ADAS_Q6e,ADAS_Q7a,ADAS_Q7b,ADAS_Q7c,ADAS_Q7d,ADAS_Q7e,ADAS_Q7f,ADAS_Q7g,ADAS_Q7h,ADAS_Q8_WL1_REC_W1_nurse,ADAS_Q8_WL1_REC_W2_magazine,ADAS_Q8_WL1_REC_W3_wizard,ADAS_Q8_WL1_REC_W4_van,ADAS_Q8_WL1_REC_W5_leopard,ADAS_Q8_WL1_REC_W6_sale,ADAS_Q8_WL1_REC_W7_sea,ADAS_Q8_WL1_REC_W8_train,ADAS_Q8_WL1_REC_W9_coin,ADAS_Q8_WL1_REC_W10_ship,ADAS_Q8_WL1_REC_W11_institution,ADAS_Q8_WL1_REC_W12_map,ADAS_Q8_WL1_REC_W13_axe,ADAS_Q8_WL1_REC_W14_board,ADAS_Q8_WL1_REC_W15_carrot,ADAS_Q8_WL1_REC_W16_milk,ADAS_Q8_WL1_REC_W17_volume,ADAS_Q8_WL1_REC_W18_forest,ADAS_Q8_WL1_REC_W19_anchor,ADAS_Q8_WL1_REC_W20_gem,ADAS_Q8_WL1_REC_W21_cat,ADAS_Q8_WL1_REC_W22_fund,ADAS_Q8_WL1_REC_W23_edge,ADAS_Q8_WL1_REC_W24_cake,ADAS_Q8_WL1_Reminder_W1_nurse,

In [52]:
len(ITEM[ITEM['VISCODE']=='bl'])

763

In [107]:
ITEM['VISCODE'].unique()

array(['bl', 'm06', 'm12', 'm18', 'm24', 'm30', 'm36'], dtype=object)

### Mini-Mental State Examination (MMSE) [ADNI1,GO,2,3] 

A Mini-Mental State Examination (MMSE) is a set of **11 questions** that doctors and other healthcare professionals commonly use to check for cognitive impairment (problems with thinking, communication, understanding and memory).

The MMSE can be used to assess 6 areas of mental abilities, including:

- Orientation to time and place — knowing the date and where you are
- Attention / concentration
- Short-term memory (recall)
- Language skills
- Visuospatial abilities — visual and spatial relationships between objects
- Ability to understand and follow instructions

The MMSE test consists of a series of tasks such as:

- Correctly identifying the current day of the week, followed by the date, the month, the season and the year 
- Correctly identifying where you are 
- Memorising a few objects and then repeating the list back later
- Writing a short sentence that is grammatically correct, such as “The dog sat on the floor”
- Copying a drawing

A 1 would indicate that the answer is correct, whereas a 2 indicates that it is incorrect. The verbatim response is in another column and it is marked with the suffix CM.

More info in: https://www.healthdirect.gov.au/mini-mental-state-examination-mmse

In [53]:
MMSE = pd.read_csv("Neuropsychological/MMSE.csv",low_memory=False)
MMSE.head()

,Phase,ID,RID,SITEID,VISCODE,VISCODE2,USERDATE,USERDATE2,EXAMDATE,MMDATE,MMDATECM,MMYEAR,MMYEARCM,MMMONTH,MMMNTHCM,MMDAY,MMDAYCM,MMSEASON,MMSESNCM,MMHOSPIT,MMHOSPCM,MMFLOOR,MMFLRCM,MMCITY,MMCITYCM,MMAREA,MMAREACM,MMSTATE,MMSTCM,MMRECALL,MMBALL,MMFLAG,MMTREE,MMTRIALS,MMD,MMDLTR,MML,MMLLTR,MMR,MMRLTR,MMO,MMOLTR,MMW,MMWLTR,MMBALLDL,MMFLAGDL,MMTREEDL,MMWATCH,MMPENCIL,MMREPEAT,MMHAND,MMFOLD,MMONFLR,MMREAD,MMWRITE,MMDRAW,MMSCORE,DONE,MMLTR1,MMLTR2,MMLTR3,MMLTR4,MMLTR5,MMLTR6,MMLTR7,WORD1,WORD1DL,WORD2,WORD2DL,WORD3,WORD3DL,WORDLIST,WORLDSCORE,DATE,SOURCE,update_stamp
0,ADNI1,10,2,107,sc,sc,2005-08-17,NaN,2005-08-17,1.0,17,1.0,2005,1.0,August,1.0,Wednesday,1.0,Summer,1.0,Memory and Aging,1.0,ground,1.0,St. Louis,1.0,Mississippi,1.0,MO,NaN,1.0,1.0,1.0,1.0,1.0,d,1.0,l,1.0,r,1.0,o,1.0,w,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,28.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005-08-17 00:00:00.0
1,ADNI1,12,1,10,f,f,2005-08-18,NaN,2005-08-18,1.0,18th,1.0,2005,1.0,August,1.0,Thursday,1.0,summer,1.0,University of Penn,1.0,1st,1.0,Philadelphia,1.0,Philadelphia,1.0,PA,NaN,1.0,1.0,1.0,1.0,1.0,d,1.0,l,1.0,r,1.0,o,1.0,w,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,28.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005-08-18 00:00:00.0
2,ADNI1,14,3,107,sc,sc,2005-08-18,NaN,2005-08-18,2.0,IDK,1.0,2005,2.0,september,2.0,IDK,2.0,fall,1.0,wash u,2.0,4th,1.0,st. louis,1.0,mississippi,1.0,mo,NaN,1.0,1.0,1.0,1.0,1.0,d,1.0,l,1.0,r,2.0,w,2.0,--,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005-08-18 00:00:00.0
3,ADNI1,16,4,10,sc,sc,2005-08-18,NaN,2005-08-18,1.0,18,1.0,2005,1.0,Agosto,1.0,Jueves,1.0,Verano,1.0,UPENN,1.0,primer piso,1.0,Phila,1.0,Phila,1.0,Penn,NaN,1.0,1.0,1.0,1.0,1.0,o,1.0,d,1.0,n,1.0,u,1.0,m,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,27.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005-08-18 00:00:00.0
4,ADNI1,18,5,107,sc,sc,2005-08-23,NaN,2005-08-23,1.0,23rd,1.0,2005,1.0,august,1.0,tuesday,1.0,summer,1.0,bjh,1.0,1st,1.0,st. louis,1.0,misissippi,1.0,mo,NaN,1.0,1.0,1.0,1.0,1.0,d,1.0,l,1.0,r,1.0,o,1.0,w,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,29.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005-08-23 00:00:00.0


In [112]:
len(MMSE[MMSE['VISCODE2']=='sc'])

3470

In [113]:
MMSE['VISCODE2'].unique()

array(['sc', 'f', 'm06', 'm12', 'm18', 'm36', 'm24', 'm48', 'm60', 'm72',
       'm84', 'm96', 'm108', 'm120', nan, 'm132', 'm78', 'm66', 'm54',
       'm126', 'm42', 'm138', 'm144', 'm90', 'm150', 'm102', 'm156',
       'm162', 'm168', 'm30', 'm114', 'm174', 'm180', 'm186', 'm192',
       'm198'], dtype=object)

### Modified Hachinski Ischemia Scale [ADNI1,GO,2,3] 

The modified Hachinski ischaemic scale is an attempt to differentiate Alzheimer's type dementia and multi-infarct dementia. Patients are scored according to certain clinical features. Patients scoring 7 or greater are classified as having “multi-infarct dementia”.

In [64]:
MODHACH = pd.read_csv("Neuropsychological/MODHACH.csv")
MODHACH.head()

,Phase,ID,RID,SITEID,VISCODE,VISCODE2,USERDATE,USERDATE2,EXAMDATE,HMONSET,HMSTEPWS,HMSOMATC,HMEMOTIO,HMHYPERT,HMSTROKE,HMNEURSM,HMNEURSG,HMSCORE,DATE,SOURCE,update_stamp
0,ADNI1,6,2,107,sc,sc,2005-08-17,NaN,2005-08-17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,2005-08-17 00:00:00.0
1,ADNI1,8,3,107,sc,sc,2005-08-18,NaN,2005-08-18,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,NaN,NaN,2005-08-18 00:00:00.0
2,ADNI1,10,4,10,sc,sc,2005-08-18,NaN,2005-08-18,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,NaN,NaN,2005-08-18 00:00:00.0
3,ADNI1,12,5,107,sc,sc,2005-08-23,NaN,2005-08-23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,2005-08-23 00:00:00.0
4,ADNI1,14,7,10,sc,sc,2005-08-26,NaN,2005-08-25,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,NaN,NaN,2005-08-26 00:00:00.0


In [110]:
len(MODHACH[MODHACH['VISCODE2']=='sc'])

3137

In [111]:
MODHACH['VISCODE2'].unique()

array(['sc', 'f', nan], dtype=object)

###  Montreal Cognitive Assessment (MoCA) [ADNIGO,2,3] 

The Montreal Cognitive Assessment (MoCA) was developed by Dr Ziad Nasreddine in Montreal, Canada in 1995 for the detection of mild cognitive impairment (MCI) by health professionals. 

The MoCA assesses several cognitive domains. These are **Visuospatial/Executive, Naming, Memory, Attention, Language, Abstraction, Delayed Recall and Orientation (to time and place)**. Many of the elements are familiar or similar to other tests of cognitive function. Visuospatial abilities are assessed using a clock-drawing task and a trail-making task which is said to be useful in assessing fitness to drive. Attention, concentration and working memory are evaluated using a sustained attention task (target detection using tapping), a serial subtraction task and digits forward and backward.

The assessment consists of a 30 point test on a single side of A4 and can be administered in 10min. A score of 26 or above is considered normal. 

**Source paper**: John Hobson, The Montreal Cognitive Assessment (MoCA), Occupational Medicine, Volume 65, Issue 9, December 2015, Pages 764–765, https://doi.org/10.1093/occmed/kqv078

In [56]:
MOCA = pd.read_csv("Neuropsychological/MOCA.csv")
MOCA.head()

,Phase,ID,RID,SITEID,VISCODE,VISCODE2,USERDATE,USERDATE2,TRAILS,CUBE,CLOCKCON,CLOCKNO,CLOCKHAN,LION,RHINO,CAMEL,IMMT1W1,IMMT1W2,IMMT1W3,IMMT1W4,IMMT1W5,IMMT2W1,IMMT2W2,IMMT2W3,IMMT2W4,IMMT2W5,DIGFOR,DIGBACK,LETTERS,SERIAL1,SERIAL2,SERIAL3,SERIAL4,SERIAL5,REPEAT1,REPEAT2,FFLUENCY,ABSTRAN,ABSMEAS,DELW1,DELW2,DELW3,DELW4,DELW5,DATE,MONTH,YEAR,DAY,PLACE,CITY,MOCA,MCDATE,SOURCE,update_stamp
0,ADNIGO,4,292,24,m48,m48,2010-05-27,NaN,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,5.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,NaN,NaN,NaN,2012-11-16 09:41:13.0
1,ADNIGO,6,1326,12,m36,m36,2010-05-30,2011-08-04,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,7.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,NaN,NaN,NaN,2012-11-16 09:41:13.0
2,ADNIGO,8,376,17,m48,m48,2010-06-02,NaN,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0,0.0,0.0,0.0,2.0,3.0,0.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,2012-11-16 09:41:13.0
3,ADNIGO,10,1117,12,m36,m36,2010-06-27,2011-11-08,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,11.0,1.0,0.0,3.0,3.0,3.0,0.0,3.0,0.0,0.0,1.0,0.0,1.0,1.0,NaN,NaN,NaN,2012-11-16 09:41:13.0
4,ADNIGO,12,618,21,m48,m48,2010-06-28,NaN,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,12.0,1.0,1.0,2.0,1.0,3.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,2012-11-16 09:41:13.0


In [114]:
len(MOCA[MOCA['VISCODE2']=='bl'])

1606

In [115]:
MOCA['VISCODE2'].unique()

array(['m48', 'm36', 'bl', 'm60', 'm06', 'm12', 'm72', 'm24', 'm84',
       'm96', 'm108', 'm120', 'm132', 'm66', 'm54', 'm78', 'm126', 'm42',
       nan, 'm138', 'm144', 'm90', 'm150', 'm102', 'm18', 'm156', 'm162',
       'm30', 'm168', 'm114', 'm174', 'm180', 'm186', 'm192', 'm198'],
      dtype=object)

### Neuropsychiatric Inventory (NPI) [ADNI2,3] 

The evaluation is trough yes (1) or no (0) questions related with depression, anxiety...

Some examples are: 

- Does {P} have periods of tearfulness or sobbing that seem to indicate sadness?
- Does {P} put him/herself down or say that he/she feels like a failure?
- Does {P} say that he/she is a bad person or deserves to be punished?
- Anxiety:  Does {P} become upset when separated from you? Does he/she have any other signs of nervousness, such as shortness of breath, sighing, being unable to relax, or feeling excessively tense?
- etc.

In [67]:
NPI = pd.read_csv("Neuropsychological/NPI.csv",low_memory=False)
NPI.head()

,Phase,ID,RID,SITEID,VISCODE,VISCODE2,USERDATE,USERDATE2,EXAMDATE,NPISOURC,SPID,NPIA,NPIA1,NPIA2,NPIA3,NPIA4,NPIA5,NPIA6,NPIA7,NPIA8,NPIA9,NPIA10A,NPIA10B,NPIA10C,NPIATOT,NPIB,NPIB1,NPIB2,NPIB3,NPIB4,NPIB5,NPIB6,NPIB7,NPIB8A,NPIB8B,NPIB8C,NPIBTOT,NPIC,NPIC1,NPIC2,NPIC3,NPIC4,NPIC5,NPIC6,NPIC7,NPIC8,NPIC9A,NPIC9B,NPIC9C,NPICTOT,NPID,NPID1,NPID2,NPID3,NPID4,NPID5,NPID6,NPID7,NPID8,NPID9A,NPID9B,NPID9C,NPIDTOT,NPIE,NPIE1,NPIE2,NPIE3,NPIE4,NPIE5,NPIE6,NPIE7,NPIE8A,NPIE8B,NPIE8C,NPIETOT,NPIF,NPIF1,NPIF2,NPIF3,NPIF4,NPIF5,NPIF6,NPIF7,NPIF8A,NPIF8B,NPIF8C,NPIFTOT,NPIG,NPIG1,NPIG2,NPIG3,NPIG4,NPIG5,NPIG6,NPIG7,NPIG8,NPIG9A,NPIG9B,NPIG9C,NPIGTOT,NPIH,NPIH1,NPIH2,NPIH3,NPIH4,NPIH5,NPIH6,NPIH7,NPIH8A,NPIH8B,NPIH8C,NPIHTOT,NPII,NPII1,NPII2,NPII3,NPII4,NPII5,NPII6,NPII7,NPII8A,NPII8B,NPII8C,NPIITOT,NPIJ,NPIJ1,NPIJ2,NPIJ3,NPIJ4,NPIJ5,NPIJ6,NPIJ7,NPIJ8A,NPIJ8B,NPIJ8C,NPIJTOT,NPIK,NPIK1,NPIK2,NPIK3,NPIK4,NPIK5,NPIK6,NPIK7,NPIK8,NPIK9A,NPIK9B,NPIK9C,NPIKTOT,NPIL,NPIL1,NPIL2,NPIL3,NPIL4,NPIL5,NPIL6,NPIL7,NPIL8,NPIL9A,NPIL9B,NPIL9C,NPILTOT,NPITOTAL,update_stamp
0,ADNI2,2,172,38,v06,m60,03/22/2011,NaN,2011-03-14,1.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,2012-11-16 11:18:17.0
1,ADNI2,4,160,38,v06,m60,03/24/2011,NaN,2011-03-15,1.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,2012-11-16 11:18:17.0
2,ADNI2,6,294,21,v06,m60,03/25/2011,NaN,2011-03-25,1.0,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,3.0,1.0,1.0,3.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,2.0,2.0,3.0,4.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,8.0,2012-11-16 11:18:17.0
3,ADNI2,8,1203,58,v06,m48,03/29/2011,NaN,2011-03-23,1.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,2012-11-16 11:18:17.0
4,ADNI2,12,1271,43,v06,m48,03/31/2011,NaN,2011-03-30,1.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0,3.0,0.0,NaN,NaN,NaN,NaN

In [116]:
len(NPI[NPI['VISCODE2']=='bl'])

1469

In [117]:
NPI['VISCODE2'].unique()

array(['m60', 'm48', 'bl', 'm12', 'm72', 'm24', 'm84', 'm36', 'm96',
       'm108', 'm120', 'm132', 'm66', 'm54', 'm78', 'm126', 'm42', 'm138',
       'm144', 'm90', nan, 'm150', 'm156', 'm102', 'm18', 'm162', 'm168',
       'm30', 'm174', 'm114', 'm180', 'm186', 'm192', 'm198'],
      dtype=object)

### Neuropsychiatric Inventory Questionnaire (NPI-Q) [ADNI1,GO,2,3] 

The Neuropsychiatric Inventory–Questionnaire (NPI-Q) was developed and cross- validated with the standard NPI to provide a brief **assessment of neuropsychiatric symptomatology**. The evaluation is trough yes (1) or no (0) questions -begining in the `NPIA` column. 

For example: 

- Does {P} believe that others are stealing from him/her, or planning to harm him/her in some way?
- Does {P} act as if he/she hears voices? Does he/she talk to people who are not there?

In addition to each question there is another column that indicates the severity. These columns are marked with the suffix SEV. 

In [69]:
NPIQ = pd.read_csv("Neuropsychological/NPIQ.csv",low_memory=False)
NPIQ.head()

,Phase,ID,RID,SITEID,VISCODE,VISCODE2,USERDATE,USERDATE2,EXAMDATE,NPISOURCE,NPIA,NPIASEV,NPIB,NPIBSEV,NPIC,NPICSEV,NPID,NPIDSEV,NPIE,NPIESEV,NPIF,NPIFSEV,NPIG,NPIGSEV,NPIH,NPIHSEV,NPII,NPIISEV,NPIJ,NPIJSEV,NPIK,NPIKSEV,NPIL,NPILSEV,NPISCORE,NPDATE,update_stamp
0,ADNI1,4,2,107,bl,bl,2005-09-09,NaN,2005-09-08,1.0,0.0,-4.0,0.0,-4.0,0.0,-4.0,0.0,-4.0,0.0,-4.0,0.0,-4.0,0.0,-4.0,0.0,-4.0,0.0,-4.0,0.0,-4.0,0.0,-4.0,0.0,-4.0,0.0,NaN,2005-09-09 00:00:00.0
1,ADNI1,6,3,107,bl,bl,2005-09-13,NaN,2005-09-12,1.0,0.0,-4.0,0.0,-4.0,0.0,-4.0,0.0,-4.0,0.0,-4.0,0.0,-4.0,1.0,1.0,0.0,-4.0,0.0,-4.0,0.0,-4.0,0.0,-4.0,0.0,-4.0,1.0,NaN,2005-09-13 00:00:00.0
2,ADNI1,8,5,107,bl,bl,2005-09-14,NaN,2005-09-07,1.0,0.0,-4.0,0.0,-4.0,0.0,-4.0,0.0,-4.0,0.0,-4.0,0.0,-4.0,0.0,-4.0,0.0,-4.0,0.0,-4.0,0.0,-4.0,0.0,-4.0,0.0,-4.0,0.0,NaN,2005-09-14 00:00:00.0
3,ADNI1,10,8,107,bl,bl,2005-09-26,NaN,2005-09-19,2.0,0.0,-4.0,0.0,-4.0,0.0,-4.0,0.0,-4.0,0.0,-4.0,0.0,-4.0,0.0,-4.0,0.0,-4.0,0.0,-4.0,0.0,-4.0,0.0,-4.0,0.0,-4.0,0.0,NaN,2005-09-26 00:00:00.0
4,ADNI1,12,7,10,bl,bl,2005-10-13,NaN,2005-10-06,1.0,0.0,-4.0,0.0,-4.0,0.0,-4.0,1.0,1.0,1.0,1.0,0.0,-4.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,10.0,NaN,2005-10-13 00:00:00.0


In [118]:
len(NPIQ[NPIQ['VISCODE2']=='bl'])

948

In [119]:
NPIQ['VISCODE2'].unique()

array(['bl', 'm06', 'm12', 'm18', 'uns1', 'm24', 'm36', 'm48', 'm60',
       'm72', 'm54', 'm66', 'm78', 'm30', 'm90', 'm42', 'm102', 'm114',
       'm126', 'm138', 'm144', 'm84', 'm96', 'm150', 'm156', 'm0', 'm162',
       'm168', 'm108', 'm120', 'm174', 'm180', 'm186', 'm192', 'm132',
       'm204', 'm198'], dtype=object)

### Neuropsychological Battery [ADNI1,GO,2,3] 

The Neuropsychological Assessment Battery (NAB; Stern & White, 2003) is a comprehensive test battery that assesses five cognitive domains (Attention, Language, Memory, Spatial, and Executive Functions). 

There are many missing data. 

In [58]:
NEUROBAT = pd.read_csv("Neuropsychological/NEUROBAT.csv",low_memory=False)
NEUROBAT.head()

,Phase,ID,RID,SITEID,VISCODE,VISCODE2,USERDATE,USERDATE2,EXAMDATE,CLOCKCIRC,CLOCKSYM,CLOCKNUM,CLOCKHAND,CLOCKTIME,CLOCKSCOR,COPYCIRC,COPYSYM,COPYNUM,COPYHAND,COPYTIME,COPYSCOR,LMSTORY,LIMMTOTAL,LIMMEND,AVTOT1,AVERR1,AVTOT2,AVERR2,AVTOT3,AVERR3,AVTOT4,AVERR4,AVTOT5,AVERR5,AVTOT6,AVERR6,AVTOTB,AVERRB,AVENDED,DSPANFOR,DSPANFLTH,DSPANBAC,DSPANBLTH,CATANIMSC,CATANPERS,CATANINTR,CATVEGESC,CATVGPERS,CATVGINTR,TRAASCOR,TRAAERRCOM,TRAAERROM,TRABSCOR,TRABERRCOM,TRABERROM,DIGITSCOR,LDELBEGIN,LDELTOTAL,LDELCUE,BNTND,BNTSPONT,BNTSTIM,BNTCSTIM,BNTPHON,BNTCPHON,BNTTOTAL,AVDELBEGAN,AVDEL30MIN,AVDELERR1,AVDELTOT,AVDELERR2,ANARTND,ANARTERR,MINTSEMCUE,MINTTOTAL,MINTUNCUED,RAVLT_forgetting,RAVLT_immediate,RAVLT_learning,RAVLT_perc_forgetting,DATE,SOURCE,update_stamp
0,ADNI1,12,2,107,sc,sc,2005-08-17,NaN,2005-08-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,0.0,-4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-08-26 16:22:44.0
1,ADNI1,14,3,107,sc,sc,2005-08-18,NaN,2005-08-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,-4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-08-26 16:22:44.0
2,ADNI1,16,4,10,sc,sc,2005-08-18,NaN,2005-08-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,0.0,-4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-08-26 16:22:44.0
3,ADNI1,18,5,107,sc,sc,2005-08-23,NaN,2005-08-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,0.0,-4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-08-26 16:22:44.0
4,ADNI1,20,7,10,sc,sc,2005-08-25,NaN,2005-08-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,-4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-08-26 16:22:44.0


In [121]:
len(NEUROBAT[NEUROBAT['VISCODE2']=='bl']) + len(NEUROBAT[NEUROBAT['VISCODE2']=='sc'])

5938

In [120]:
NEUROBAT['VISCODE2'].unique()

array(['sc', 'f', 'bl', 'm06', 'm12', 'm36', 'uns1', 'm18', 'm24', 'm48',
       'm60', 'm72', 'm84', 'm96', 'm108', 'm120', nan, 'm132', 'm78',
       'm66', 'm54', 'm126', 'm42', 'm138', 'm144', 'm90', 'm150', 'm102',
       'm156', 'm162', 'm30', 'm168', 'm114', 'm174', 'm180', 'm186',
       'm192', 'm198'], dtype=object)

### UW - Neuropsych Summary Scores [ADNI1,GO,2,3] 

We derived composite scores for **executive functioning** (ADNI-EF) and **memory** (ADNI-MEM) using data from the ADNI neuropsychological battery using item response theory (IRT) methods. The formation of ADNI-MEM was complicated by the use of different word lists in the Rey Auditory Verbal Learning Test (RAVLT) and the ADAS-Cog, and by Logical Memory I data missing by design. ADNI-EF and ADNI-MEM have been validated in published papers1,2.

- Compositive score for executive functioning (`ADNI-EF`)
- Composite score for memory (`ADNI_MEM`)
- Composite language score (`ADNI_LAN`)
- Composite score for visuospatial functioning (`ADNI_VS`)

In [61]:
UW = pd.read_csv("Neuropsychological/UWNPSYCHSUM_01_23_23.csv",low_memory=False)
UW.head()

,COLPROT,RID,VISCODE,VISCODE2,USERDATE,EXAMDATE,ADNI_MEM,ADNI_EF,ADNI_LAN,ADNI_VS,ADNI_EF2,update_stamp
0,ADNI1,2,m06,m06,2006-03-06,2006-03-06,0.581,0.088,0.216,-0.984,-0.487,2023-01-24 14:52:11.0
1,ADNI1,2,m36,m36,2008-08-27,2008-08-27,0.405,0.088,0.417,0.706,-0.287,2023-01-24 14:52:11.0
2,ADNI1,2,bl,bl,2005-09-09,2005-09-08,0.486,0.091,0.106,0.706,-0.309,2023-01-24 14:52:11.0
3,ADNI2,2,v21,m96,2013-09-12,2013-09-09,0.325,-0.790,0.340,0.706,-0.965,2023-01-24 14:52:11.0
4,ADNI2,2,m72,m72,2011-09-20,2011-09-19,0.350,0.343,0.237,-0.104,-0.081,2023-01-24 14:52:11.0


In [63]:
len(UW[UW['VISCODE2']=='bl'])

2425

In [122]:
UW['VISCODE2'].unique()

array(['m06', 'm36', 'bl', 'm96', 'm72', 'm60', 'm120', 'm84', 'm132',
       'm108', 'm144', 'm12', 'm24', 'm18', 'm48', 'm174', 'm168', 'm150',
       'm162', 'm156', 'm186', 'm180', 'm138', 'm198', 'm192', 'm126',
       'm102', 'm78', 'm90', 'm114', 'm66', 'm54', 'm42', 'm30'],
      dtype=object)